    base_folder_path = "/content/drive/MyDrive/nomu_dataset3" # <<< 원본 데이터 폴더 경로 확인!
    result_dir = "/content/drive/MyDrive/nomu_rag_result"    # <<< 결과 저장 폴더 경로 확인!

# 0: 필요한 라이브러리 설치 (OpenAI 추가)

In [1]:
# === 단계 0: 필요한 라이브러리 설치 (의존성 충돌 해결 버전 + OpenAI 추가) ===
print("--- 단계 0: 라이브러리 설치 시작 (OpenAI 추가) ---")
!pip install -qU \
    langchain langchain-core langchain-community langchain-openai openai \
    pypdf openpyxl xlrd unstructured faiss-cpu sentence-transformers \
    pdf2image pillow pdfminer.six rank_bm25 pillow-heif jq \
    google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread \
    ragas datasets \
    pandas==2.2.2 \
    PyPDF2 \
    fsspec==2025.3.2 # <<< fsspec 버전은 환경에 따라 조정 필요, 원래 버전 사용

# google-ai-generativelanguage는 OpenAI 사용 시 필수는 아님 (필요시 유지)
# !pip install -qU google-ai-generativelanguage==0.6.15

print("\n[알림] 라이브러리 설치/업데이트 완료. langchain-openai, openai 추가됨.")

--- 단계 0: 라이브러리 설치 시작 (OpenAI 추가) ---
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6

# 1: 기본 및 필요 라이브러리 임포트 (OpenAI LLM 임포트 추가)

In [2]:
# === 단계 1: 기본 및 필요 라이브러리 임포트 ===

import os
import glob
import subprocess
import sys
import warnings
import time
import pandas as pd
from google.colab import drive, auth, userdata
import PyPDF2 # 명시적 임포트
import json
import pickle
import torch
import numpy as np
from tqdm.notebook import tqdm
import gspread
from google.auth import default as google_auth_default
from datasets import Dataset
import re
import traceback # 오류 상세 출력을 위해 추가

# LangChain 관련 임포트
from langchain_core.documents import Document
from langchain_community.document_loaders import (
    PyPDFLoader, UnstructuredExcelLoader, CSVLoader,
    UnstructuredFileLoader, DirectoryLoader, GoogleDriveLoader
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_google_genai import GoogleGenerativeAIEmbeddings # Google 임베딩 (필요 시 유지)
from langchain_community.embeddings import HuggingFaceEmbeddings # HuggingFace 임베딩 (유지)
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
# === LLM 임포트 변경 ===
# from langchain_google_genai import ChatGoogleGenerativeAI # <<< 삭제 또는 주석 처리
from langchain_openai import ChatOpenAI # <<< OpenAI LLM 클래스 임포트
# =======================
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# RAGAS 관련 임포트 (기존 유지)
try:
    from ragas import evaluate
    from ragas.metrics import context_precision, context_recall, faithfulness, answer_relevancy
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper
except ImportError:
    print("!! Ragas 관련 라이브러리가 설치되지 않았습니다. 평가 단계 전에 설치가 필요합니다.")
    LangchainLLMWrapper = None; LangchainEmbeddingsWrapper = None; context_precision = None; context_recall = None; faithfulness = None; answer_relevancy = None

# 기타 평가 관련 임포트 (기존 유지)
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
# import google.generativeai as genai # OpenAI 사용 시 필수는 아님

warnings.filterwarnings("ignore") # 경고 메시지 숨기기

print("--- 단계 1: 라이브러리 임포트 완료 (ChatOpenAI 임포트됨) ---")

--- 단계 1: 라이브러리 임포트 완료 (ChatOpenAI 임포트됨) ---


# 2: 환경 설정 (OpenAI API 키 추가)

    base_folder_path = "/content/drive/MyDrive/nomu_dataset3" # <<< 원본 데이터 폴더 경로 확인!
    result_dir = "/content/drive/MyDrive/nomu_rag_result"    # <<< 결과 저장 폴더 경로 확인!
    # <<< nomu_dataset3 폴더의 실제 ID로 변경하거나 확인! >>>
예)
    https://drive.google.com/drive/u/0/folders/1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U

target_folder_id = "1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U"

In [3]:
# === 단계 2: 환경 설정 (Drive 마운트, API 키, 경로) ===
print("\n--- 단계 2: 환경 설정 시작 ---")

# --- Google Drive 마운트 (기존 유지) ---
DRIVE_MOUNTED = False
try:
    drive.mount('/content/drive', force_remount=False)
    DRIVE_MOUNTED = True
    print("[성공] Google Drive 마운트 완료.")
except Exception as e:
    print(f"[실패] Google Drive 마운트 오류: {e}")

# --- Google 인증 (Colab 사용자 인증 - 필요시 유지) ---
# Google Drive Loader 등을 사용한다면 인증 유지 필요
try:
    auth.authenticate_user()
    print("[성공] Google Colab 사용자 인증 완료.")
except Exception as e:
    print(f"[실패] Google Colab 인증 오류: {e}")

# === API 키 설정 변경 ===
# --- OpenAI API 키 설정 ---
OPENAI_API_KEY = None
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # Colab Secrets 우선 확인
    if not OPENAI_API_KEY: OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') # 환경 변수 확인
    if not OPENAI_API_KEY: raise ValueError("OpenAI API 키를 Colab Secrets 또는 환경 변수에서 찾을 수 없습니다.")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    print("[성공] OpenAI API 키 로드 및 설정 완료.")
except Exception as e:
    print(f"[실패] OpenAI API 키 로드/설정 오류: {e}")
    print("   !! OpenAI API 키 없이는 이후 LLM, RAGAS 평가 등 사용 불가 !!")

# --- Google AI API 키 설정 (선택 사항) ---
# 만약 Google Embedding 등을 계속 사용한다면 유지, 아니면 제거 가능
# GOOGLE_API_KEY = None
# try:
#     GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
#     if not GOOGLE_API_KEY: GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
#     if GOOGLE_API_KEY:
#         os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
#         import google.generativeai as genai
#         genai.configure(api_key=GOOGLE_API_KEY)
#         print("[정보] Google API 키 로드 및 설정 완료 (선택 사항).")
#     # else: print("[정보] Google API 키 로드 안 됨 (선택 사항).") # 키 없어도 오류 아님
# except Exception as e: print(f"[경고] Google API 키 설정 중 오류 발생 (선택 사항): {e}")
# =========================

# --- 경로 설정 (기존 유지) ---
if DRIVE_MOUNTED:
    base_folder_path = "/content/drive/MyDrive/nomu_dataset3"
    result_dir = "/content/drive/MyDrive/nomu_rag_result"
else:
    base_folder_path = "./nomu_data_local"
    result_dir = "./nomu_rag_result_local"

print(f"데이터 소스 검색 경로: {base_folder_path}")
print(f"결과 저장 경로: {result_dir}")
os.makedirs(base_folder_path, exist_ok=True)
os.makedirs(result_dir, exist_ok=True)

# Google Sheets 폴더 ID (필요시 유지)
target_folder_id = "1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U"
print(f"Google Sheets 검색 대상 폴더 ID: {target_folder_id}")

# --- 결과 파일 경로 (기존 유지) ---
vs_status_file = os.path.join(result_dir, "vectorstore_build_status.json")
vs_checkpoint_path = os.path.join(result_dir, "faiss_index_nomu_checkpoint")
vs_final_save_path = os.path.join(result_dir, "faiss_index_nomu_final")
bm25_data_save_path = os.path.join(result_dir, "split_texts_for_bm25.pkl")

print("--- 단계 2: 환경 설정 완료 (OpenAI API 키 설정됨) ---")


--- 단계 2: 환경 설정 시작 ---
Mounted at /content/drive
[성공] Google Drive 마운트 완료.
[성공] Google Colab 사용자 인증 완료.
[성공] OpenAI API 키 로드 및 설정 완료.
데이터 소스 검색 경로: /content/drive/MyDrive/nomu_dataset3
결과 저장 경로: /content/drive/MyDrive/nomu_rag_result
Google Sheets 검색 대상 폴더 ID: 1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U
--- 단계 2: 환경 설정 완료 (OpenAI API 키 설정됨) ---


# 3: 데이터 로딩

In [4]:
# === 단계 3: 데이터 로딩  ===

print("\n--- 단계 3: 데이터 로딩 시작 ---")

loaded_documents = []
loading_errors = {}

# --- 3.1. Google Sheets 로딩 ---
print("\n--- 3.1 Google Sheets 파일 로딩 ---")
if not target_folder_id or "YOUR_" in target_folder_id:
    print("  ⚠️ 경고: Google Drive 폴더 ID가 유효하지 않아 Google Sheet 로딩 건너<0xEB><0x9A><A9>니다.")
elif not DRIVE_MOUNTED:
     print("  ⚠️ 경고: Google Drive가 마운트되지 않아 Google Sheet 로딩 건너<0xEB><0x9A><A9>니다.")
else:
    try:
        gsheet_loader = GoogleDriveLoader(folder_id=target_folder_id, file_types=["sheet"], recursive=True)
        print(f"  GoogleDriveLoader로 폴더 '{target_folder_id}' 로딩 중...")
        gsheet_docs = gsheet_loader.load()
        if gsheet_docs:
            print(f"  [성공] Google Sheets 로딩 ({len(gsheet_docs)}개 조각).")
            for doc in gsheet_docs:
                doc.metadata['file_type'] = 'google_sheet'
                if 'source' not in doc.metadata and 'id' in doc.metadata:
                    doc.metadata['source'] = f"https://docs.google.com/spreadsheets/d/{doc.metadata['id']}"
            loaded_documents.extend(gsheet_docs)
        else: print("  [정보] 해당 폴더에 Google Sheets 파일 없음.")
    except ImportError as ie: print(f"  ❌ 임포트 오류: {ie}. 관련 라이브러리 설치 필요."); loading_errors['google_sheets'] = str(ie)
    except Exception as e: error_msg = f"{type(e).__name__}: {e}"; print(f"  ❌ 로딩 오류: {error_msg}"); loading_errors['google_sheets'] = error_msg

# --- 3.2. 다른 파일 형식 로딩 (PDF, Excel, CSV, TXT 등) ---
print("\n--- 3.2 다른 파일 형식 로딩 (DirectoryLoader) ---")
if not DRIVE_MOUNTED and not os.path.exists(base_folder_path):
    print(f"  ⚠️ 경고: Drive 미마운트 및 로컬 경로({base_folder_path}) 없음. 파일 로딩 건너<0xEB><0x9A><A9>니다.")
else:
    LOADER_MAPPING = { ".pdf": (PyPDFLoader, {}), ".xlsx": (UnstructuredExcelLoader, {"mode": "single"}), ".xls": (UnstructuredExcelLoader, {"mode": "single"}), ".csv": (CSVLoader, {"encoding": "utf-8"}), ".txt": (UnstructuredFileLoader, {}) }
    supported_extensions = list(LOADER_MAPPING.keys())
    for ext in supported_extensions:
        print(f"\n  '{ext}' 확장자 로딩 ({base_folder_path})...")
        loader_cls, loader_args = LOADER_MAPPING[ext]
        try:
            loader = DirectoryLoader( base_folder_path, glob=f"**/*{ext}", loader_cls=loader_cls, loader_kwargs=loader_args, recursive=True, show_progress=True, use_multithreading=True, silent_errors=False )
            docs = loader.load()
            if docs:
                print(f"    [성공] '{ext}' 로딩 ({len(docs)}개 조각).")
                for doc in docs: doc.metadata['file_type'] = ext.lstrip('.')
                loaded_documents.extend(docs)
            else: print(f"    [정보] '{ext}' 파일 없음.")
        except ImportError as ie: error_msg = f"{ie}"; print(f"    ❌ 임포트 오류: {error_msg}"); loading_errors[f'loader_{ext}'] = f"ImportError: {error_msg}"
        except Exception as e: error_msg = f"{type(e).__name__}: {e}"; print(f"    ❌ 로딩 오류: {error_msg}"); loading_errors[f'loader_{ext}'] = error_msg

# --- 3.3. 로딩 결과 요약 ---
print(f"\n--- 최종 로드된 문서 조각 수: {len(loaded_documents)} ---")
doc_type_counts = {}
for doc in loaded_documents: file_type = doc.metadata.get('file_type', 'unknown'); doc_type_counts[file_type] = doc_type_counts.get(file_type, 0) + 1
print("\n--- 로드된 문서 타입별 개수 ---");
if doc_type_counts: [print(f"  - {f_type}: {count}개 조각") for f_type, count in sorted(doc_type_counts.items())]
else: print("  로드된 문서 없음.")
if loading_errors: print("\n--- 로딩 오류 요약 ---"); [print(f"  - {src}: {err}") for src, err in loading_errors.items()]
if loaded_documents:
    print("\n--- 첫 로드 문서 샘플 ---")
    try: first_doc = loaded_documents[0]; print(f"  타입: {first_doc.metadata.get('file_type')}\n  메타데이터: {first_doc.metadata}\n  내용(200자): {first_doc.page_content[:200]}...")
    except Exception as e: print(f"  !! 샘플 출력 오류: {e}")
else: print("\n로드된 문서 없음. 경로, 파일 형식, 권한 확인 필요.")
print("\n--- 단계 3: 데이터 로딩 완료 ---")



--- 단계 3: 데이터 로딩 시작 ---

--- 3.1 Google Sheets 파일 로딩 ---
  GoogleDriveLoader로 폴더 '1FIA_HGeYbRVaH_USuxUdjhFAoReUTo9U' 로딩 중...
  [성공] Google Sheets 로딩 (358개 조각).

--- 3.2 다른 파일 형식 로딩 (DirectoryLoader) ---

  '.pdf' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


100%|██████████| 122/122 [00:31<00:00,  3.86it/s]


    [성공] '.pdf' 로딩 (1712개 조각).

  '.xlsx' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


0it [00:00, ?it/s]


    [정보] '.xlsx' 파일 없음.

  '.xls' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


...

    [정보] '.xls' 파일 없음.

  '.csv' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


0it [00:00, ?it/s]


    [정보] '.csv' 파일 없음.

  '.txt' 확장자 로딩 (/content/drive/MyDrive/nomu_dataset3)...


0it [00:00, ?it/s]

    [정보] '.txt' 파일 없음.

--- 최종 로드된 문서 조각 수: 2070 ---

--- 로드된 문서 타입별 개수 ---
  - google_sheet: 358개 조각
  - pdf: 1712개 조각

--- 첫 로드 문서 샘플 ---
  타입: google_sheet
  메타데이터: {'source': 'https://docs.google.com/spreadsheets/d/1QeMvmrcYe6QQ8L1n6o1NQ7ASTPafmzdbuDAAKXFCu3k/edit?gid=0', 'title': 'filtered_qa_dataset - Sheet1', 'row': 1, 'file_type': 'google_sheet'}
  내용(200자): number: 0
question: 근로계약이 미성년자에게 불리하다고 인정되는 경우 미성년후견인은 그 계약을 해지할 수 있나요?
answer: 네. 근로계약을 해지할 수 있습니다.
ground_truths: ["「근로기준법」 제67조 제2항은 "친권자","후견인 또는 고용노동부장관은 근로계약이 미성년자에게 불리하다고 인정하는 경우에는 이를 해지할 수 있다....

--- 단계 3: 데이터 로딩 완료 ---


# 4: 텍스트 분할 (Chunking, 패턴 기반 + 길이 제한)

In [5]:
# === 단계 4: 텍스트 분할 (패턴 기반 + 길이 제한) ===
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document # Document 클래스 임포트 확인

print("\n--- 단계 4: 텍스트 분할 시작 (패턴 기반 적용) ---")

split_texts = []
chunk_size_setting = 500
chunk_overlap_setting = 100
# 최대 길이 초과 시 사용할 fallback 스플리터
fallback_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size_setting,
    chunk_overlap=chunk_overlap_setting
)

if 'loaded_documents' in locals() and loaded_documents:
    print(f"[정보] 패턴 기반 청킹 시도 (법률 PDF 대상). 기준 크기={chunk_size_setting}, 중첩={chunk_overlap_setting}")
    processed_docs_count = 0
    skipped_docs_count = 0

    for doc in tqdm(loaded_documents, desc="문서 청킹 중"):
        doc_content = doc.page_content
        doc_metadata = doc.metadata
        file_type = doc_metadata.get('file_type', 'unknown')

        # --- PDF 문서(법률 문서로 간주)에만 패턴 기반 적용 ---
        # 파일 타입이 'pdf'가 아니거나 내용이 없으면 기본 스플리터 사용
        if file_type != 'pdf' or not doc_content.strip():
            if doc_content.strip(): # 내용이 있을 때만 분할
                 try:
                     sub_chunks = fallback_splitter.split_text(doc_content)
                     for chunk_text in sub_chunks:
                         split_texts.append(Document(page_content=chunk_text, metadata=doc_metadata.copy()))
                     processed_docs_count += 1
                 except Exception as e:
                      print(f"\n!! 기본 분할 오류 (타입: {file_type}, 소스: {doc_metadata.get('source', 'N/A')}): {e}")
                      skipped_docs_count += 1
            else:
                skipped_docs_count += 1
            continue # 다음 문서로 이동

        # --- 패턴 기반 청킹 로직 (PDF 대상) ---
        try:
            # 1. "제X조" 패턴으로 먼저 크게 나누기
            # 정규표현식: '제' + (선택적 공백) + 숫자 + (선택적 공백) + '조'
            # re.split은 구분자도 결과에 포함시키므로, 이를 활용하여 조 제목을 유지
            preliminary_chunks_with_title = re.split(r'(제\s?\d+\s?조)', doc_content)

            current_chunk_content = ""
            # 첫 부분 처리 (첫 '제X조' 이전 내용)
            if preliminary_chunks_with_title[0].strip():
                 current_chunk_content = preliminary_chunks_with_title[0].strip()

            # '제X조' 제목과 그 내용을 묶어서 처리
            for i in range(1, len(preliminary_chunks_with_title), 2):
                title = preliminary_chunks_with_title[i] # 예: "제1조"
                content = preliminary_chunks_with_title[i+1] if (i+1) < len(preliminary_chunks_with_title) else ""

                article_block = title + content # "제1조 내용..."

                # 이전 청크가 있고 + 현재 조항을 합쳐도 크기를 넘지 않으면 합침
                # (짧은 조항들이 합쳐지는 효과)
                if current_chunk_content and (len(current_chunk_content) + len(article_block) <= chunk_size_setting):
                    current_chunk_content += "\n\n" + article_block # 문단 구분 추가
                else:
                    # 이전 청크가 너무 길었거나, 합치면 길어지는 경우
                    # 이전 청크를 최종 처리하고 현재 조항으로 새 청크 시작
                    if current_chunk_content: # 이전 청크 내용이 있으면
                        # 이전 청크가 최대 크기를 넘는지 확인
                        if len(current_chunk_content) > chunk_size_setting:
                            # 넘으면 fallback 스플리터로 재분할
                            sub_chunks = fallback_splitter.split_text(current_chunk_content)
                            for chunk_text in sub_chunks:
                                split_texts.append(Document(page_content=chunk_text, metadata=doc_metadata.copy()))
                        else:
                            # 안 넘으면 그대로 추가
                            split_texts.append(Document(page_content=current_chunk_content, metadata=doc_metadata.copy()))

                    # 현재 조항으로 새 청크 시작
                    current_chunk_content = article_block.strip() # 새 청크 시작

            # 마지막 남은 청크 처리
            if current_chunk_content:
                 if len(current_chunk_content) > chunk_size_setting:
                     sub_chunks = fallback_splitter.split_text(current_chunk_content)
                     for chunk_text in sub_chunks:
                         split_texts.append(Document(page_content=chunk_text, metadata=doc_metadata.copy()))
                 else:
                     split_texts.append(Document(page_content=current_chunk_content, metadata=doc_metadata.copy()))

            processed_docs_count += 1

        except Exception as e:
            print(f"\n!! 패턴 기반 분할 오류 (소스: {doc_metadata.get('source', 'N/A')}): {e}")
            # 오류 발생 시 해당 문서는 기본 스플리터로 처리 시도 (선택 사항)
            try:
                sub_chunks = fallback_splitter.split_text(doc_content)
                for chunk_text in sub_chunks:
                     split_texts.append(Document(page_content=chunk_text, metadata=doc_metadata.copy()))
                print(f"  -> 오류 발생하여 기본 분할 방식으로 처리함.")
                processed_docs_count += 1
            except Exception as fallback_e:
                 print(f"  -> 기본 분할 방식도 실패: {fallback_e}")
                 skipped_docs_count += 1

    print(f"\n[성공] 총 {processed_docs_count}개 원본 조각 처리 -> {len(split_texts)}개 청크 분할 완료.")
    if skipped_docs_count > 0: print(f"[경고] {skipped_docs_count}개 원본 조각 처리 중 오류 발생하여 건너<0xEB><0x9B><0x81>.")

    if split_texts:
         print("\n첫 분할 청크 샘플:")
         print(f"  내용(200자): {split_texts[0].page_content[:200]}...")
         print(f"  메타데이터: {split_texts[0].metadata}")
         # 크기 확인 (Fallback 적용 후에도 클 수 있음)
         oversized = [len(c.page_content) for c in split_texts if len(c.page_content) > chunk_size_setting + chunk_overlap_setting] # 오버랩 고려
         if oversized: print(f"\n[경고] {len(oversized)}개 청크가 최대 크기({chunk_size_setting}자)를 약간 초과할 수 있음 (최대 {max(oversized)}자).")
         else: print(f"\n[정보] 모든 청크 크기 비교적 정상.")
    else: print("[정보] 생성된 청크 없음.")

else:
    print("!! 분할할 로드된 문서 없음.")

print("--- 단계 4: 텍스트 분할 완료 ---")


--- 단계 4: 텍스트 분할 시작 (패턴 기반 적용) ---
[정보] 패턴 기반 청킹 시도 (법률 PDF 대상). 기준 크기=500, 중첩=100


문서 청킹 중:   0%|          | 0/2070 [00:00<?, ?it/s]


[성공] 총 2070개 원본 조각 처리 -> 9512개 청크 분할 완료.

첫 분할 청크 샘플:
  내용(200자): number: 0
question: 근로계약이 미성년자에게 불리하다고 인정되는 경우 미성년후견인은 그 계약을 해지할 수 있나요?
answer: 네. 근로계약을 해지할 수 있습니다.
ground_truths: ["「근로기준법」 제67조 제2항은 "친권자","후견인 또는 고용노동부장관은 근로계약이 미성년자에게 불리하다고 인정하는 경우에는 이를 해지할 수 있다....
  메타데이터: {'source': 'https://docs.google.com/spreadsheets/d/1QeMvmrcYe6QQ8L1n6o1NQ7ASTPafmzdbuDAAKXFCu3k/edit?gid=0', 'title': 'filtered_qa_dataset - Sheet1', 'row': 1, 'file_type': 'google_sheet'}

[정보] 모든 청크 크기 비교적 정상.
--- 단계 4: 텍스트 분할 완료 ---


# 5: 임베딩 모델 설정 (Hugging Face: skt/kobert-base-v1)

In [6]:
!pip install -U sentence-transformers transformers huggingface-hub --quiet

In [7]:
pip install -U langchain-openai openai

# --- 한국어 임베딩 모델 설정 (KR-BERT 계열 모델 선택) ---
# 다양한 한국어 BERT 기반 모델 사용 가능 (성능/리소스 고려하여 선택)
# 예시 모델:
# 1. 'skt/kobert-base-v1': SKT KoBERT (널리 사용됨)
# 2. 'kykim/bert-kor-base': 김기현님 BERT 모델
# 3. 'jhgan/ko-sbert-nli': 한국어 문장 유사도 학습 모델 (SBERT 계열, 추천)
# 4. 'snunlp/KR-BERT-char16424': 서울대 KR-BERT (문자 단위)
# 5. 'beomi/kcbert-base': Beomi님 KcBERT

In [8]:
# === 단계 5: 임베딩 모델 설정 (OpenAI: text-embedding-ada-002) ===
print("\n--- 단계 5: 임베딩 모델 설정 시작 (OpenAI: text-embedding-ada-002) ---")

# 필요한 라이브러리 임포트
try:
    # OpenAI 임베딩을 위한 클래스 임포트
    # pip install -U langchain-openai  <-- 먼저 이 패키지를 설치해야 합니다.
    from langchain_openai import OpenAIEmbeddings
    print("[정보] langchain_openai.OpenAIEmbeddings 임포트 확인.")
except ImportError as e:
    print(f"!! [오류] 필요한 라이브러리 임포트 실패: {e}")
    print("   langchain-openai, openai 패키지가 설치되었는지 확인하세요.")
    # 필요한 클래스가 없으면 이후 진행 불가
    OpenAIEmbeddings = None

embeddings = None # 최종 임베딩 객체 변수 초기화

if OpenAIEmbeddings: # 라이브러리 임포트 성공 시 진행
    try:
        # 사용할 OpenAI 모델 지정
        openai_model_name = "text-embedding-ada-002"
        print(f"[정보] OpenAI 임베딩 모델 ({openai_model_name}) 설정 시도...")
        print("[정보] 단계 2에서 설정된 OPENAI_API_KEY 환경 변수를 사용합니다.")

        # <<< OpenAIEmbeddings 클래스 사용 (API 키 자동 감지) >>>
        # 단계 2에서 os.environ["OPENAI_API_KEY"] 가 설정되었으므로,
        # API 키를 명시적으로 전달할 필요 없이 자동으로 사용됩니다.
        embeddings = OpenAIEmbeddings(
            model=openai_model_name # 사용할 모델 이름만 지정
            # openai_api_key 파라미터 생략
        )
        # ------------------------------------------------------

        # 간단한 테스트 (API 키 유효성 및 통신 확인)
        print("[정보] 임베딩 모델 테스트 중 (OpenAI API 호출)...")
        _ = embeddings.embed_query("테스트 문장입니다.")
        print(f"[성공] OpenAI 임베딩 모델 ({openai_model_name}) 설정 및 테스트 완료.")

    except Exception as e:
        print(f"!! [오류] OpenAI 임베딩 모델 ({openai_model_name}) 설정 실패: {e}")
        print("   - 단계 2에서 OpenAI API 키가 올바르게 설정되었는지 확인하세요.") # <<< 오류 메시지 수정
        print("   - OpenAI API 키 자체의 유효성 및 할당량(quota)을 확인하세요.")
        print("   - 인터넷 연결 상태 및 OpenAI 서비스 상태를 확인하세요.")
        print("   - 관련 라이브러리(langchain-openai, openai) 설치 및 호환성을 확인하세요.")
        embeddings = None # 실패 시 None으로 설정
else:
    print("!! 필요한 라이브러리(OpenAIEmbeddings) 임포트 실패. 임베딩 모델 설정 불가.")

print("--- 단계 5: 임베딩 모델 설정 완료 ---")

# OpenAI 모델은 외부 API를 사용하므로 로컬 GPU/CPU 확인은 불필요합니다.


--- 단계 5: 임베딩 모델 설정 시작 (OpenAI: text-embedding-ada-002) ---
[정보] langchain_openai.OpenAIEmbeddings 임포트 확인.
[정보] OpenAI 임베딩 모델 (text-embedding-ada-002) 설정 시도...
[정보] 단계 2에서 설정된 OPENAI_API_KEY 환경 변수를 사용합니다.
[정보] 임베딩 모델 테스트 중 (OpenAI API 호출)...
[성공] OpenAI 임베딩 모델 (text-embedding-ada-002) 설정 및 테스트 완료.
--- 단계 5: 임베딩 모델 설정 완료 ---


# 단계 6: Vector Store 구축 (FAISS)

--- 6.2: 경로 설정 ---
📂 결과 저장 경로: /content/drive/MyDrive/nomu_rag_result
  - 상태 파일: /content/drive/MyDrive/nomu_rag_result/vectorstore_build_status.json
  - 체크포인트 폴더 (FAISS): /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_checkpoint
  - 최종 인덱스 폴더 (FAISS): /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_final
  - BM25용 데이터 파일: /content/drive/MyDrive/nomu_rag_result/split_texts_for_bm25.pkl

In [9]:
# -*- coding: utf-8 -*-
# === 단계 6: Vector Store 구축 (FAISS) 및 BM25용 데이터 저장 ===
# 체크포인팅 방식을 save_local로 변경

import time
import os
import json
# import pickle # pickle은 BM25 데이터 저장에만 사용
import torch
import traceback
from tqdm.auto import tqdm
# LangChain 관련 임포트 확인
if 'FAISS' not in locals() or 'Document' not in locals():
    from langchain_community.vectorstores import FAISS
    from langchain_core.documents import Document
    print("⚠️ FAISS 또는 Document 클래스 재임포트됨.")
if 'HuggingFaceEmbeddings' not in locals() and 'GoogleGenerativeAIEmbeddings' not in locals():
    # 사용 중인 임베딩 클래스를 임포트해야 합니다.
    from langchain_community.embeddings import HuggingFaceEmbeddings # 예시
    # from langchain_google_genai import GoogleGenerativeAIEmbeddings # 예시
    print("⚠️ 임베딩 클래스 재임포트됨.")


print("\n--- 단계 6: Vector Store 구축 및 BM25 데이터 저장 시작 ---")

# --- 6.1: 설정 및 입력 변수 확인 ---
print("\n--- 6.1: 설정 및 입력 변수 확인 ---")
vectorstore = None
batch_size = 500 # 배치 크기 확인
sleep_time = 5
max_retries = 3
total_chunks = 0
start_index = 0
loaded_from_checkpoint = False

if 'split_texts' not in locals() or not isinstance(split_texts, list) or not split_texts: print("!! 오류: 'split_texts' 없음. 중단."); exit()
elif 'embeddings' not in locals() or embeddings is None: print("!! 오류: 'embeddings' 없음. 중단."); exit()
else: total_chunks = len(split_texts); print(f"✅ 입력 확인: 총 {total_chunks}개 청크 및 임베딩 모델 확인됨.")

# --- 6.2: 경로 설정 및 디렉토리 생성 ---
print("\n--- 6.2: 경로 설정 ---")
if 'result_dir' not in locals() or not result_dir: result_dir = "/content/drive/MyDrive/nomu_rag_result"; print(f"⚠️ result_dir 변수 없어 기본 경로 설정: {result_dir}"); os.makedirs(result_dir, exist_ok=True)
else: print(f"📂 결과 저장 경로: {result_dir}"); os.makedirs(result_dir, exist_ok=True)

vs_status_file = os.path.join(result_dir, "vectorstore_build_status.json")
# <<< 체크포인트 경로를 폴더로 변경 (save_local 사용) >>>
vs_checkpoint_path = os.path.join(result_dir, "faiss_index_nomu_checkpoint") # .pkl 대신 폴더명
vs_final_save_path = os.path.join(result_dir, "faiss_index_nomu_final")
bm25_data_save_path = os.path.join(result_dir, "split_texts_for_bm25.pkl")

print(f"  - 상태 파일: {vs_status_file}")
print(f"  - 체크포인트 폴더 (FAISS): {vs_checkpoint_path}") # <<< 이름 변경
print(f"  - 최종 인덱스 폴더 (FAISS): {vs_final_save_path}")
print(f"  - BM25용 데이터 파일: {bm25_data_save_path}")

# --- 6.3: GPU 확인 --- (이전과 동일)
print("\n--- 6.3: 장치 확인 ---")
device = 'cuda' if torch.cuda.is_available() else 'cpu'; print(f"ℹ️ 사용할 장치: {device}");
if device == 'cpu': print("⚠️ 경고: GPU 사용 불가. 시간 소요 예상.")

# --- 6.4: 이전 작업 상태 및 체크포인트 로드 ---
if total_chunks > 0:
    print("\n--- 6.4: 이전 작업 상태 및 체크포인트 로드 ---")
    try:
        # <<< 체크포인트 파일 대신 폴더 존재 여부 확인 >>>
        if os.path.exists(vs_status_file) and os.path.isdir(vs_checkpoint_path):
            with open(vs_status_file, 'r') as f_status: status_data = json.load(f_status); last_processed_index = status_data.get('last_processed_index', -1); start_index = last_processed_index + 1; print(f"  - 상태 로드 완료. 마지막 인덱스: {last_processed_index}")

            # <<< pickle.load 대신 FAISS.load_local 사용 >>>
            print(f"  - 체크포인트 인덱스 로딩: {vs_checkpoint_path}")
            if 'embeddings' in locals() and embeddings:
                vectorstore = FAISS.load_local(
                    folder_path=vs_checkpoint_path,
                    embeddings=embeddings,
                    allow_dangerous_deserialization=True # 신뢰할 수 있는 소스일 때만
                )
                print(f"  - FAISS 체크포인트 로드 완료. {vectorstore.index.ntotal} 벡터 포함.")
                loaded_from_checkpoint = True
            else:
                print("  !! 오류: 임베딩 함수('embeddings')가 없어 체크포인트를 로드할 수 없음. 처음부터 시작.")
                vectorstore = None; start_index = 0; loaded_from_checkpoint = False

            if start_index >= total_chunks: print("  ✅ 이전 FAISS 작업 완료됨.")
            elif loaded_from_checkpoint: print(f"  ▶️ {start_index}번 인덱스부터 FAISS 작업 재개.")
            # else 블록은 위에서 처리됨

        else: print(f"  - 이전 상태 파일 또는 체크포인트 폴더 없음. 처음부터 시작."); vectorstore = None; start_index = 0
    except Exception as e: print(f"  ⚠️ 상태/체크포인트 로드 오류: {e}. 처음부터 시작."); traceback.print_exc(); vectorstore = None; start_index = 0
else: print("ℹ️ 처리할 청크 없음.")

# --- 6.5: FAISS Vector Store 구축 ---
if total_chunks > 0 and start_index < total_chunks:
    print(f"\n--- 6.5: FAISS 인덱스 구축 시작 (총 {total_chunks} 중 {start_index}부터, 배치 {batch_size}) ---")
    all_processed_successfully = True
    try:
        progress_bar = tqdm(range(start_index, total_chunks, batch_size), initial=start_index // batch_size, total=-(total_chunks // -batch_size), desc="FAISS 구축 중")
        for i in progress_bar:
            batch_start_idx = i; batch_end_idx = min(i + batch_size, total_chunks); batch_docs = split_texts[batch_start_idx:batch_end_idx]
            for attempt in range(max_retries):
                try:
                    if i == start_index and not loaded_from_checkpoint:
                        progress_bar.set_description(f"첫 배치({batch_start_idx}-{batch_end_idx-1}) 생성 중")
                        vectorstore = FAISS.from_documents(batch_docs, embeddings)
                        print(f"\n   - 첫 배치 FAISS 생성 완료 (벡터 {vectorstore.index.ntotal}개)")
                    elif vectorstore is not None:
                        progress_bar.set_description(f"배치({batch_start_idx}-{batch_end_idx-1}) 추가 중")
                        vectorstore.add_documents(batch_docs)
                        print(f"\n   - 배치 FAISS 추가 완료 (총 벡터 {vectorstore.index.ntotal}개)")
                    else: raise ValueError("Vectorstore 객체 None 상태.")

                    current_processed_index = batch_end_idx - 1
                    try:
                        # <<< 체크포인팅: 상태 저장 후 save_local 호출 >>>
                        with open(vs_status_file, 'w') as f_status: json.dump({'last_processed_index': current_processed_index}, f_status)
                        # 체크포인트 폴더에 덮어쓰기 (이전 파일 삭제됨)
                        vectorstore.save_local(vs_checkpoint_path)
                        # print(f"  💾 체크포인트 저장 완료 (폴더: {vs_checkpoint_path}, 인덱스: {current_processed_index})")
                    except Exception as e_save: print(f"  ⚠️ 진행 상황(체크포인트) 저장 실패: {e_save}")
                    break # 성공 시 재시도 루프 탈출
                except Exception as e:
                    print(f"\n!! 배치 처리 오류 (시도 {attempt+1}/{max_retries}): {type(e).__name__} - {e}")
                    traceback.print_exc()
                    if attempt < max_retries - 1: wait_time = sleep_time * (attempt + 2); progress_bar.set_description(f"오류, {wait_time}초 후 재시도..."); print(f"  ... {wait_time}초 후 재시도 ..."); time.sleep(wait_time)
                    else: print("!! 최대 재시도 초과. 구축 중단."); all_processed_successfully = False; raise RuntimeError(f"최대 재시도 실패: {e}")
            if not all_processed_successfully: break
            if batch_end_idx < total_chunks: time.sleep(sleep_time) # API Rate Limit

        # --- 루프 완료 후 최종 처리 ---
        if all_processed_successfully:
            print("\n[성공] 모든 FAISS 배치 처리 완료.")
            # --- 6.6: 최종 결과 저장 ---
            print("\n--- 6.6: 최종 결과 저장 ---")
            # 1. 최종 FAISS 인덱스 저장 (save_local)
            print(f"💾 최종 FAISS 인덱스 저장: {vs_final_save_path}")
            try:
                if vectorstore: vectorstore.save_local(vs_final_save_path); print(f"✅ 최종 FAISS 인덱스 저장 완료.")
                else: print("⚠️ 최종 저장 시 VectorStore 객체 없음.")
            except Exception as e: print(f"❌ 최종 FAISS 인덱스 저장 오류: {e}"); traceback.print_exc()
            # 2. BM25용 데이터 저장 (pickle)
            print(f"💾 BM25용 텍스트 데이터 저장: {bm25_data_save_path}")
            try:
                if 'split_texts' in locals() and split_texts:
                    # <<< pickle 임포트 확인 >>>
                    import pickle
                    with open(bm25_data_save_path, 'wb') as f_texts: pickle.dump(split_texts, f_texts); print(f"✅ BM25용 텍스트 데이터 ({len(split_texts)}개 청크) 저장 완료.")
                else: print("⚠️ BM25용 데이터('split_texts') 없어 저장 불가.")
            except Exception as e: print(f"❌ split_texts 저장 실패: {e}"); traceback.print_exc()
            # (선택) 중간 파일 삭제
            # try: ... (삭제 로직) ... except ...

    except Exception as e_main_loop: print(f"\n!! FAISS 구축 실패: {e_main_loop}"); print(f"[정보] 마지막 성공 지점 데이터는 '{vs_checkpoint_path}' 폴더에 있을 수 있음.")

# --- 이전 실행에서 이미 완료된 경우 처리 ---
elif total_chunks > 0 and start_index >= total_chunks:
     print("\n✅ FAISS Vector Store 구축 작업이 이미 완료된 상태입니다.")
     # 완료된 상태에서도 최종 파일들이 존재하는지 확인하고 없으면 저장 시도
     # 1. 최종 FAISS 인덱스 확인 및 저장
     if not os.path.isdir(vs_final_save_path): # <<< 폴더 존재 여부 확인 >>>
         if vectorstore: # 체크포인트에서 로드된 vectorstore가 있다면
             print(f"\n💾 최종 FAISS 인덱스 폴더 없어 저장 시도: '{vs_final_save_path}'") # <<< print 문 분리
             # <<< try...except 블록을 새 줄에서 시작 >>>
             try:
                 vectorstore.save_local(vs_final_save_path)
                 print("✅ 저장 완료.")
             except Exception as e:
                 print(f"❌ 저장 오류: {e}")
                 traceback.print_exc()
         else:
             print(f"⚠️ 최종 FAISS 인덱스 폴더 없고, 로드된 vectorstore도 없어 저장 불가.")

     # 2. BM25용 데이터 확인 및 저장
     if not os.path.exists(bm25_data_save_path):
         if 'split_texts' in locals() and split_texts:
             print(f"\n💾 BM25용 텍스트 데이터 파일({bm25_data_save_path}) 없어 저장 시도...") # <<< print 문 분리
             # <<< try...except 블록을 새 줄에서 시작 >>>
             try:
                 # <<< pickle 임포트 확인 (필요시) >>>
                 import pickle
                 with open(bm25_data_save_path, 'wb') as f:
                     pickle.dump(split_texts, f)
                 print("✅ 저장 완료.")
             except Exception as e:
                 print(f"❌ 저장 실패: {e}")
                 traceback.print_exc()
         else:
             print(f"⚠️ BM25용 데이터 파일 없고, split_texts 변수도 없어 저장 불가.")

print("\n--- 단계 6: Vector Store 처리 및 BM25 데이터 저장 완료 ---")


--- 단계 6: Vector Store 구축 및 BM25 데이터 저장 시작 ---

--- 6.1: 설정 및 입력 변수 확인 ---
✅ 입력 확인: 총 9512개 청크 및 임베딩 모델 확인됨.

--- 6.2: 경로 설정 ---
📂 결과 저장 경로: /content/drive/MyDrive/nomu_rag_result
  - 상태 파일: /content/drive/MyDrive/nomu_rag_result/vectorstore_build_status.json
  - 체크포인트 폴더 (FAISS): /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_checkpoint
  - 최종 인덱스 폴더 (FAISS): /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_final
  - BM25용 데이터 파일: /content/drive/MyDrive/nomu_rag_result/split_texts_for_bm25.pkl

--- 6.3: 장치 확인 ---
ℹ️ 사용할 장치: cpu
⚠️ 경고: GPU 사용 불가. 시간 소요 예상.

--- 6.4: 이전 작업 상태 및 체크포인트 로드 ---
  - 이전 상태 파일 또는 체크포인트 폴더 없음. 처음부터 시작.

--- 6.5: FAISS 인덱스 구축 시작 (총 9512 중 0부터, 배치 500) ---


FAISS 구축 중:   0%|          | 0/20 [00:00<?, ?it/s]


   - 첫 배치 FAISS 생성 완료 (벡터 500개)

   - 배치 FAISS 추가 완료 (총 벡터 1000개)

   - 배치 FAISS 추가 완료 (총 벡터 1500개)

   - 배치 FAISS 추가 완료 (총 벡터 2000개)

   - 배치 FAISS 추가 완료 (총 벡터 2500개)

   - 배치 FAISS 추가 완료 (총 벡터 3000개)

   - 배치 FAISS 추가 완료 (총 벡터 3500개)

   - 배치 FAISS 추가 완료 (총 벡터 4000개)

   - 배치 FAISS 추가 완료 (총 벡터 4500개)

   - 배치 FAISS 추가 완료 (총 벡터 5000개)

   - 배치 FAISS 추가 완료 (총 벡터 5500개)

   - 배치 FAISS 추가 완료 (총 벡터 6000개)

   - 배치 FAISS 추가 완료 (총 벡터 6500개)

   - 배치 FAISS 추가 완료 (총 벡터 7000개)

   - 배치 FAISS 추가 완료 (총 벡터 7500개)

   - 배치 FAISS 추가 완료 (총 벡터 8000개)

   - 배치 FAISS 추가 완료 (총 벡터 8500개)

   - 배치 FAISS 추가 완료 (총 벡터 9000개)

   - 배치 FAISS 추가 완료 (총 벡터 9500개)

   - 배치 FAISS 추가 완료 (총 벡터 9512개)

[성공] 모든 FAISS 배치 처리 완료.

--- 6.6: 최종 결과 저장 ---
💾 최종 FAISS 인덱스 저장: /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_final
✅ 최종 FAISS 인덱스 저장 완료.
💾 BM25용 텍스트 데이터 저장: /content/drive/MyDrive/nomu_rag_result/split_texts_for_bm25.pkl
✅ BM25용 텍스트 데이터 (9512개 청크) 저장 완료.

--- 단계 6: Vector Store 처리 및 BM25 데이터 저장 완료 ---


# 단계 7: 저장된 인덱스 및 데이터 로드 (재실행 시 사용)

⏰**임베딩 모델은 로드에 필요하므로 다시 정의 (단계 5와 동일해야 함) **

이 블록은 최초 실행 시에는 실행할 필요가 없으나,

노트북을 다시 열고 단계 6까지의 결과를 재사용하고 싶을 때 실행합니다.

In [ ]:
# === 단계 7: 저장된 인덱스 및 데이터 로드 (재실행 시 사용) ===

# 이 블록은 최초 실행 시에는 실행할 필요가 없으나,
# 노트북을 다시 열고 단계 6까지의 결과를 재사용하고 싶을 때 실행합니다.
print("\n--- 단계 7: 저장된 인덱스 및 데이터 로드 시도 ---")

loaded_vectorstore = None # 로드된 FAISS 객체
loaded_split_texts = None # 로드된 BM25용 텍스트

# 임베딩 모델은 로드에 필요하므로 다시 정의 (단계 5와 동일해야 함)
print("⏳ 로드에 필요한 임베딩 모델 재설정 중...")
load_embeddings = None
if 'embeddings' in locals() and embeddings: # 이전 단계에서 성공적으로 로드되었다면 재사용
    load_embeddings = embeddings
    print("[정보] 이전 단계의 임베딩 모델 사용.")
else: # 아니라면 새로 로드 시도
    try:
        hf_model_name = "snunlp/KR-BERT-char16424"
        hf_encode_kwargs = {'normalize_embeddings': True}
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        load_embeddings = HuggingFaceEmbeddings(model_name=hf_model_name, model_kwargs={'device': device}, encode_kwargs=hf_encode_kwargs)
        _ = load_embeddings.embed_query("로드 테스트")
        print(f"[성공] 로드용 임베딩 모델 로딩 완료 ({hf_model_name}, 장치: {device}).")
    except Exception as e:
        print(f"!! [오류] 로드용 임베딩 모델 로딩 실패: {e}")

# FAISS 인덱스 로드
if load_embeddings is not None:
    if os.path.exists(vs_final_save_path):
        print(f"💾 저장된 FAISS 인덱스 로딩 중: {vs_final_save_path}")
        try:
            loaded_vectorstore = FAISS.load_local(folder_path=vs_final_save_path, embeddings=load_embeddings, allow_dangerous_deserialization=True)
            print(f"✅ FAISS 인덱스 로드 완료 ({loaded_vectorstore.index.ntotal}개 벡터).")
        except Exception as e: print(f"❌ FAISS 인덱스 로드 오류: {e}"); print(traceback.format_exc())
    else: print(f"❌ 저장된 FAISS 인덱스 경로 없음: {vs_final_save_path}")
else: print("⚠️ 임베딩 모델 준비 안 됨. FAISS 인덱스 로드 불가.")

# BM25용 데이터 로드
if os.path.exists(bm25_data_save_path):
    print(f"💾 저장된 BM25용 텍스트 데이터 로딩 중: {bm25_data_save_path}")
    try:
        with open(bm25_data_save_path, 'rb') as f: loaded_split_texts = pickle.load(f)
        print(f"✅ BM25용 텍스트 데이터 로드 완료 ({len(loaded_split_texts)}개 청크).")
    except Exception as e: print(f"❌ BM25 데이터 로드 오류: {e}"); traceback.print_exc()
else: print(f"❌ 저장된 BM25용 텍스트 데이터 경로 없음: {bm25_data_save_path}")

print("--- 단계 7: 저장된 인덱스 및 데이터 로드 완료 ---")



--- 단계 7: 저장된 인덱스 및 데이터 로드 시도 ---
⏳ 로드에 필요한 임베딩 모델 재설정 중...


config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/397M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/397M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/104k [00:00<?, ?B/s]

[성공] 로드용 임베딩 모델 로딩 완료 (snunlp/KR-BERT-char16424, 장치: cpu).
💾 저장된 FAISS 인덱스 로딩 중: /content/drive/MyDrive/nomu_rag_result/faiss_index_nomu_final
✅ FAISS 인덱스 로드 완료 (5010개 벡터).
💾 저장된 BM25용 텍스트 데이터 로딩 중: /content/drive/MyDrive/nomu_rag_result/split_texts_for_bm25.pkl
✅ BM25용 텍스트 데이터 로드 완료 (5010개 청크).
--- 단계 7: 저장된 인덱스 및 데이터 로드 완료 ---


# 단계 8: Retriever 설정 (하이브리드 검색)

In [10]:
# === 단계 8: Retriever 설정 (하이브리드 검색) ===

print("\n--- 단계 8: Retriever 설정 시작 (하이브리드 검색) ---")

# 사용할 VectorStore와 split_texts 결정
# 로드 성공 시 로드된 것 사용, 아니면 구축 단계에서 생성된 것 사용
final_vectorstore = None
final_split_texts = None # BM25용

if 'loaded_vectorstore' in locals() and loaded_vectorstore:
    final_vectorstore = loaded_vectorstore
    print("[정보] 로드된 FAISS VectorStore 사용.")
elif 'vectorstore' in locals() and vectorstore:
    final_vectorstore = vectorstore
    print("[정보] 새로 생성된 FAISS VectorStore 사용.")
else:
    print("!! 오류: 사용 가능한 FAISS VectorStore 객체 없음.")

if 'loaded_split_texts' in locals() and loaded_split_texts:
    final_split_texts = loaded_split_texts
    print("[정보] 로드된 split_texts 데이터 사용 (BM25용).")
elif 'split_texts' in locals() and split_texts:
    final_split_texts = split_texts
    print("[정보] 현재 세션의 split_texts 데이터 사용 (BM25용).")
else:
    print("!! 오류: BM25용 split_texts 데이터 없음.")

retriever = None # 최종 리트리버 초기화

if final_vectorstore:
    # Dense Retriever (FAISS)
    faiss_retriever = final_vectorstore.as_retriever(search_kwargs={'k': 6})
    print(f"- Dense Retriever (FAISS) 설정 완료 (k={faiss_retriever.search_kwargs.get('k')}).")

    if final_split_texts: # BM25용 데이터가 있을 때만 하이브리드 시도
        try:
            # Sparse Retriever (BM25)
            bm25_retriever = BM25Retriever.from_documents(final_split_texts)
            bm25_retriever.k = 6
            print(f"- Sparse Retriever (BM25) 설정 완료 (k={bm25_retriever.k}).")
            # Ensemble Retriever
            ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], weights=[0.4, 0.6])
            retriever = ensemble_retriever
            print(f"- Ensemble Retriever 설정 완료 (Weights: BM25=0.7, FAISS=0.3).")
        except Exception as e:
            print(f"!! BM25/Ensemble 설정 실패: {e}. Dense Retriever만 사용.")
            retriever = faiss_retriever # Fallback
    else:
        print("⚠️ BM25 데이터 없어 Dense Retriever(FAISS)만 사용합니다.")
        retriever = faiss_retriever # Fallback
else:
    print("!! Vector Store 준비 안 됨. Retriever 설정 불가.")

print("--- 단계 8: Retriever 설정 완료 ---")



--- 단계 8: Retriever 설정 시작 (하이브리드 검색) ---
[정보] 새로 생성된 FAISS VectorStore 사용.
[정보] 현재 세션의 split_texts 데이터 사용 (BM25용).
- Dense Retriever (FAISS) 설정 완료 (k=6).
- Sparse Retriever (BM25) 설정 완료 (k=6).
- Ensemble Retriever 설정 완료 (Weights: BM25=0.7, FAISS=0.3).
--- 단계 8: Retriever 설정 완료 ---


# 9: LLM 설정 (OpenAI - gpt-4.1-nano)

In [11]:
# === 단계 9: LLM 설정 (OpenAI - gpt-4.1-nano) ===

print("\n--- 단계 9: LLM 설정 시작 (OpenAI) ---")
llm = None
# OpenAI API 키 설정 여부 확인 (단계 2에서 설정됨)
if 'OPENAI_API_KEY' in os.environ and os.environ["OPENAI_API_KEY"]:
    try:
        # === 사용할 OpenAI 모델 지정 ===
        # llm_model_name = "gpt-4o" # 최신 모델 (성능과 속도 균형)
        llm_model_name = "gpt-4.1-nano" # 만약 더 작고 빠른 모델이 필요하다면 고려 (현재는 gpt-4o가 가장 유사)
        # llm_model_name = "gpt-3.5-turbo" # 속도/비용 우선 시 고려
        # ==============================

        llm = ChatOpenAI(
            model=llm_model_name,
            temperature=0 # 답변의 창의성 조절 (낮을수록 결정적)
            # max_tokens=1024 # 필요시 최대 출력 토큰 수 제한
        )
        print(f"[성공] OpenAI ({llm.model_name}) LLM 로딩 완료.") # .model 대신 .model_name 사용

    except ImportError:
         print("!! [오류] langchain-openai 또는 openai 라이브러리가 설치되지 않았습니다.")
         print("   단계 0의 설치 명령을 확인하고 런타임을 재시작하세요.")
    except Exception as e:
        print(f"!! [오류] OpenAI LLM 로딩 실패: {e}")
        traceback.print_exc() # 상세 오류 출력
else:
    print("!! [오류] OpenAI API 키가 설정되지 않았습니다. LLM 로드 불가.")

print("--- 단계 9: LLM 설정 완료 ---")


--- 단계 9: LLM 설정 시작 (OpenAI) ---
[성공] OpenAI (gpt-4.1-nano) LLM 로딩 완료.
--- 단계 9: LLM 설정 완료 ---


# 10: RAG Chain/파이프라인 구축

(((((리트리버 확인할 때만 실행)))))Retriever 반환값 확인:
retriever 자체가 빈 리스트를 반환하는 것은 아닌지 확인

In [12]:
if 'retriever' in locals() and retriever:
    test_query = "근로자 해고 예고"
    try:
        retrieved_docs = retriever.invoke(test_query)
        print(f"\n[Retriever 테스트 결과] '{test_query}'에 대해 {len(retrieved_docs)}개 문서 검색됨:")
        if retrieved_docs:
             for i, doc in enumerate(retrieved_docs[:3]): # 처음 2개만 출력
                 print(f"--- 문서 {i+1} ---")
                 print(doc.page_content[:100] + "...")
                 print(doc.metadata)
        else:
             print(" -> 검색된 문서 없음!")
    except Exception as e:
        print(f"!! Retriever 테스트 중 오류: {e}")
else:
    print("!! Retriever 객체가 없어 테스트 불가.")


[Retriever 테스트 결과] '근로자 해고 예고'에 대해 11개 문서 검색됨:
--- 문서 1 ---
제26조에 따라 해고
가 예고된 사람, 경영상 이유에 따른 사업주의 권고에 따라 퇴직이 예정된 사람 및 사업주(사업주가 법인인 경우에는
그 대표자를 말한다)의 배우자 및 직계존속ㆍ...
{'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2025-04-15T11:11:38+09:00', 'moddate': '2025-04-15T11:11:38+09:00', 'source': '/content/drive/MyDrive/nomu_dataset3/고용보험법 시행령(대통령령)(제35277호)(20250223).pdf', 'total_pages': 63, 'page': 8, 'page_label': '9', 'file_type': 'pdf'}
--- 문서 2 ---
분류: 부당해고
question: 근로자를 해고하려면 몇일 전에 예고를 해야 하나요?
관련 법령: 근로기준법
answer: 근로기준법 제26조에 따르면, 적어도 30일 전에 예고를...
{'source': 'https://docs.google.com/spreadsheets/d/192InT2H5seStldZkzDoE0IhOvNy-ATfN-jOvyy3AAEg/edit?gid=0', 'title': 'nomu_qa - 시트1', 'row': 1, 'file_type': 'google_sheet'}
--- 문서 3 ---
제26조(해고의 예고) 사용자는 근로자를 해고(경영상 이유에 의한 해고를 포함한다)하려면 적어도 30일 전에 예고를 하
여야 하고, 30일 전에 예고를 하지 아니하였을 때에는 30...
{'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2025-04-01T12:35:38+09:00', 'mo

In [13]:
# === 단계 10: RAG Chain/파이프라인 구축 ===
# (코드 변경 없음 - 단계 9에서 변경된 llm 객체를 사용)
print("\n--- 단계 10: RAG Chain 구축 시작 ---")
qa_chain = None
if llm and retriever: # llm이 ChatOpenAI 인스턴스로 준비됨
    template = """당신은 한국의 노무 규정 및 관련 문서에 기반하여 질문에 답변하는 유용한 AI 어시스턴트입니다.
    주어진 다음 컨텍스트 정보만을 사용하여 질문에 답변하십시오. 컨텍스트에 없는 내용은 답변에 포함하지 마십시오.
    만약 컨텍스트 정보만으로 질문에 답할 수 없다면, "
    제공된 문서 내용만으로는 답변할 수 없습니다."라고 명확히 답변하십시오.
    답변은 간결하고 명확하게 한국어로 작성해주세요.

    컨텍스트:
    {context}

    질문: {question}

    답변 (한국어):"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
    try:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm, # ChatOpenAI 인스턴스 사용
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
        )
        print("[성공] RetrievalQA Chain 구축 완료.")
    except Exception as e: print(f"!! RAG Chain 구축 실패: {e}")
else: print("!! LLM 또는 Retriever 준비 안 됨. RAG Chain 구축 불가.")
print("--- 단계 10: RAG Chain 구축 완료 ---")


--- 단계 10: RAG Chain 구축 시작 ---
[성공] RetrievalQA Chain 구축 완료.
--- 단계 10: RAG Chain 구축 완료 ---


# 단계 11: RAG 시스템 실행 (질의-응답)

query = "근로자를 해고하려면 몇일 전에 예고를 해야 하나요?" # 테스트 질문

In [14]:
# === 단계 11: RAG 시스템 실행 (질의-응답) ===
# (코드 변경 없음 - 단계 10에서 생성된 qa_chain 사용)
print("\n--- 단계 11: RAG 시스템 실행 시작 ---")
query = None # 결과 저장용
result = None
if qa_chain:
    query = "근로자를 해고하려면 몇일 전에 예고를 해야 하나요?" # 테스트 질문
    print(f"\n입력 질문: {query}")
    try:
        print("답변 생성 중..."); start_qa_time = time.time()
        # qa_chain은 내부적으로 ChatOpenAI를 사용하게 됨
        result = qa_chain.invoke({"query": query})
        end_qa_time = time.time(); print(f"답변 생성 완료 ({end_qa_time - start_qa_time:.2f}초)")
        print("\n[최종 답변]:"); print(result.get("result", "N/A"))
        print("\n[참고 문서 (Source Documents)]:")
        # ... (나머지 출력 코드 동일) ...
    except Exception as e: print(f"!! RAG Chain 실행 오류: {e}"); traceback.print_exc()
else: print("!! RAG Chain 준비 안 됨. 실행 불가.")
print("\n--- 단계 11: RAG 시스템 실행 완료 ---")


--- 단계 11: RAG 시스템 실행 시작 ---

입력 질문: 근로자를 해고하려면 몇일 전에 예고를 해야 하나요?
답변 생성 중...
답변 생성 완료 (1.09초)

[최종 답변]:
근로기준법 제26조에 따르면, 근로자를 해고하려면 적어도 30일 전에 예고를 하여야 합니다.

[참고 문서 (Source Documents)]:

--- 단계 11: RAG 시스템 실행 완료 ---


# 12. RAGAS Retriever 평가

사용할 Google Sheet 파일 이름

GSHEET_FILE_NAME = "filtered_qa_dataset"

QUESTION_COLUMN = "question"

ANSWER_COLUMN = "answer"             

GROUND_TRUTHS_COLUMN = "ground_truths"


상태 저장: result_dir 아래의 ragas_eval_exec_status.json (마지막 처리 인덱스 저장)

결과 저장: result_dir 아래의 ragas_eval_exec_results_partial.csv (배치별 결과 누적 저장)

전체 경로 (예시): /content/drive/MyDrive/nomu_rag_result/ragas_eval_exec_status.json

# [디버깅] 단일 항목 RAGAS 평가 테스트

In [ ]:
# --- 12.5 코드 수정 (테스트용) ---
print("\n--- [디버깅] 단일 항목 RAGAS 평가 테스트 ---")
if not qa_df.empty and retriever_available and llm_available and evaluate:
    # 첫 번째 데이터 항목 준비
    test_index = 0
    test_row = qa_df.iloc[test_index]
    question_text = str(test_row.get(QUESTION_COLUMN, "")).strip()
    answer_text = str(test_row.get(ANSWER_COLUMN, "")).strip() if answer_column_exists else ""
    ground_truths_input = test_row.get(GROUND_TRUTHS_COLUMN, "[]") if ground_truths_column_exists else "[]"
    ground_truths_list = []
    retrieved_contexts = []
    single_eval_data = None

    if question_text:
        try:
            # Ground Truths 파싱
            if ground_truths_column_exists and isinstance(ground_truths_input, str) and ground_truths_input.strip():
                try: parsed_list = json.loads(ground_truths_input); ground_truths_list = [str(item).strip() for item in parsed_list if isinstance(parsed_list, list) and item is not None and str(item).strip()] if isinstance(parsed_list, list) else [ground_truths_input.strip()]
                except: ground_truths_list = [ground_truths_input.strip()]
            elif isinstance(ground_truths_input, list): ground_truths_list = [str(item).strip() for item in ground_truths_input if item is not None and str(item).strip()]
            print(f"  [디버깅] Parsed ground_truths: {ground_truths_list}") # 파싱 결과 확인

            # Retriever 실행
            retrieved_docs = retriever.invoke(question_text)
            retrieved_contexts = [doc.page_content for doc in retrieved_docs]
            print(f"  [디버깅] Retrieved contexts count: {len(retrieved_contexts)}") # 컨텍스트 수 확인

            single_eval_data = {
                "question": question_text,
                "contexts": retrieved_contexts,
                "ground_truths": ground_truths_list,
                "ground_truth": answer_text # answer 컬럼 이름 확인 필요
            }

        except Exception as prep_e:
            print(f"!! 단일 항목 데이터 준비 중 오류: {prep_e}")

        # 단일 데이터 평가 실행
        if single_eval_data:
            try:
                print("  단일 항목 RAGAS 평가 실행 (raise_exceptions=True)...")
                single_dataset = Dataset.from_list([single_eval_data])
                metrics_to_evaluate = [context_precision, context_recall]
                if not all(metrics_to_evaluate): raise ImportError("Ragas 메트릭 임포트 실패")

                single_result = evaluate(
                    dataset=single_dataset,
                    metrics=metrics_to_evaluate,
                    llm=ragas_llm,
                    embeddings=ragas_embeddings if embeddings_available else None,
                    raise_exceptions=True # 오류 발생 시 즉시 중단하고 표시
                )
                print("  [성공] 단일 항목 평가 완료:")
                print(single_result)
            except Exception as single_eval_e:
                print(f"!! [오류] 단일 항목 RAGAS 평가 중 오류 발생:")
                traceback.print_exc() # 전체 오류 스택 출력
    else:
        print("!! 테스트할 질문이 비어있습니다.")
else:
    print("!! 단일 항목 평가 실행에 필요한 객체 또는 데이터 부족.")

# --- 원래 배치 평가는 일단 주석 처리 ---
# print("\n--- 12.5: RAGAS 평가 실행 (배치 처리 및 체크포인팅) ---")
# ... (기존 배치 평가 코드) ...


--- [디버깅] 단일 항목 RAGAS 평가 테스트 ---
  [디버깅] Parsed ground_truths: ['["「근로기준법」 제67조 제2항은 "친권자","후견인 또는 고용노동부장관은 근로계약이 미성년자에게 불리하다고 인정하는 경우에는 이를 해지할 수 있다."라고 규정하고 있습니다."]']
  [디버깅] Retrieved contexts count: 9
  단일 항목 RAGAS 평가 실행 (raise_exceptions=True)...


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

  [성공] 단일 항목 평가 완료:
{'context_precision': 0.7708, 'context_recall': 1.0000}


# 단계 12: RAGAS Retriever 평가 (체크포인팅 및 재개 기능 추가)

In [15]:
# -*- coding: utf-8 -*-
# === 단계 12: RAGAS Retriever 평가 (체크포인팅, 재개, 지연 시간 추가) ===

# --- 12.0: 필요 라이브러리 임포트 및 재확인 ---
print("\n--- 단계 12: RAGAS Retriever 평가 시작 (체크포인팅 및 지연 시간 적용) ---")
import os
import pandas as pd
import gspread
# google.colab 라이브러리는 Colab 환경에서만 사용 가능합니다.
try:
    from google.colab import auth
    print("[정보] Google Colab 환경 감지됨.")
    COLAB_ENV = True
except ImportError:
    print("[정보] Google Colab 환경 아님. 로컬 인증 필요.")
    COLAB_ENV = False
from google.auth import default as google_auth_default
import time
from tqdm.notebook import tqdm # Colab/Jupyter 환경용 진행 바
import numpy as np
import json
import warnings
import traceback # 상세 오류 출력

# RAGAS 및 관련 라이브러리 임포트
try:
    from datasets import Dataset
    from ragas import evaluate
    from ragas.metrics import context_precision, context_recall # 필요한 메트릭
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper
    print("[정보] RAGAS 및 관련 라이브러리 임포트 확인.")
    RAGAS_AVAILABLE = True
except ImportError as e:
    print(f"!! [오류] RAGAS 관련 라이브러리 임포트 실패: {e}")
    Dataset = None; evaluate = None; context_precision = None; context_recall = None;
    LangchainLLMWrapper = None; LangchainEmbeddingsWrapper = None
    RAGAS_AVAILABLE = False

warnings.filterwarnings("ignore") # 경고 메시지 숨기기 (선택 사항)

# --- 12.1: Google 인증 및 gspread 클라이언트 확인 ---
print("\n--- 12.1: Google 인증 및 gspread 클라이언트 확인 ---")
gc = None
try:
    if COLAB_ENV:
        auth.authenticate_user() # Colab 사용자 인증
    creds, _ = google_auth_default() # 기본 인증 정보 가져오기
    gc = gspread.authorize(creds)
    print("[성공] Google 인증 및 gspread 클라이언트 준비 완료.")
except Exception as e:
    print(f"!! [실패] Google 인증/gspread 오류: {e}. Google Sheet 로드 불가.")

# --- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---
# 이전에 정의된 llm, retriever, embeddings 객체가 있다고 가정합니다.
print("\n--- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---")
llm_available = 'llm' in locals() and llm is not None
retriever_available = 'retriever' in locals() and retriever is not None
embeddings_available = 'embeddings' in locals() and embeddings is not None
ragas_llm = None; ragas_embeddings = None

if RAGAS_AVAILABLE: # RAGAS 라이브러리 로드 성공 시
    if llm_available:
        if LangchainLLMWrapper:
            try:
                model_name_display = getattr(llm, 'model', getattr(llm, 'model_name', 'N/A'))
                ragas_llm = LangchainLLMWrapper(llm)
                print(f"[성공] RAGAS LLM 래퍼 초기화 (모델: {model_name_display})")
            except Exception as e:
                print(f"!! [오류] RAGAS LLM 래퍼 초기화 실패: {e}"); llm_available = False
    else: print("!! [오류] LLM 객체('llm') 없음.")

    if embeddings_available:
        if LangchainEmbeddingsWrapper:
            try:
                ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)
                print("[성공] RAGAS Embedding 래퍼 초기화 완료.")
            except Exception as e:
                print(f"!! [경고] RAGAS Embedding 래퍼 초기화 오류: {e}."); embeddings_available = False
    else: print("[경고] Embedding 객체('embeddings') 없음. 일부 메트릭 사용 불가.")
else:
    print("!! RAGAS 라이브러리 임포트 실패. RAGAS 평가 불가.")

# --- 12.3: 평가 데이터셋 로드 (Google Sheet: filtered_qa_dataset) ---
print("\n--- 12.3: 평가 데이터셋 로드 ---")
GSHEET_FILE_NAME = "filtered_qa_dataset" # 사용할 Google Sheet 파일 이름
QUESTION_COLUMN = "question"; ANSWER_COLUMN = "answer"; GROUND_TRUTHS_COLUMN = "ground_truths"
qa_df = pd.DataFrame(); ground_truths_column_exists = False; answer_column_exists = False
if gc: # Google 인증 성공 시에만 시도
    print(f"Google Sheet '{GSHEET_FILE_NAME}' 로드 시도...")
    try:
        spreadsheet = gc.open(GSHEET_FILE_NAME)
        worksheet = spreadsheet.get_worksheet(0) # 첫 번째 시트 사용
        print("  워크시트 데이터 로드 중..."); time.sleep(1) # API 호출 시간 고려
        data = worksheet.get_all_values()
        if len(data) > 1: # 헤더 포함 2줄 이상일 때
            header = data[0]
            qa_df = pd.DataFrame(data[1:], columns=header)
            print(f"  [성공] Google Sheet 로드 완료: {len(qa_df)}개 행"); print(f"     컬럼 목록: {qa_df.columns.tolist()}")
            # 필수 컬럼 및 옵션 컬럼 존재 여부 확인
            if QUESTION_COLUMN not in qa_df.columns: raise ValueError(f"필수 컬럼 '{QUESTION_COLUMN}' 없음.")
            if GROUND_TRUTHS_COLUMN in qa_df.columns:
                ground_truths_column_exists = True; print(f"  [정보] '{GROUND_TRUTHS_COLUMN}' 컬럼 확인.")
            else: print(f"  !! [경고] '{GROUND_TRUTHS_COLUMN}' 컬럼 없음 (context_recall 계산 불가).")
            if ANSWER_COLUMN in qa_df.columns:
                answer_column_exists = True; print(f"  [정보] '{ANSWER_COLUMN}' 컬럼 확인.")
            else: print(f"  !! [경고] '{ANSWER_COLUMN}' 컬럼 없음 (일부 메트릭 사용 불가).")
        else: print("  !! [오류] Google Sheet 데이터 없음 (헤더만 있거나 비어 있음)."); qa_df = pd.DataFrame()
    except gspread.exceptions.SpreadsheetNotFound: print(f"  !! [오류] Google Sheet '{GSHEET_FILE_NAME}' 찾을 수 없음."); qa_df = pd.DataFrame()
    except Exception as e: print(f"  !! [오류] Google Sheet 로드 중 오류: {e}"); traceback.print_exc(); qa_df = pd.DataFrame()
else: print("!! Google Sheet 클라이언트(gc) 준비 안 됨. 데이터 로드 불가.")

# --- 12.4: 체크포인트 설정 및 로드 (복원) ---
print("\n--- 12.4: 체크포인트 설정 및 로드 ---")

# 결과 저장 디렉토리 (이전에 정의되었다고 가정)
# 예시: result_dir = "/content/drive/MyDrive/nomu_rag_result"
result_dir_valid = False
status_file = None
partial_results_file = None
if 'result_dir' in locals() and isinstance(result_dir, str) and os.path.isdir(result_dir):
     print(f"결과 저장 경로 확인: {result_dir}")
     result_dir_valid = True
     # 체크포인트 파일 경로 설정
     status_file = os.path.join(result_dir, "ragas_eval_retriever_status.json")
     partial_results_file = os.path.join(result_dir, "ragas_eval_retriever_results_partial.csv")
     print(f"상태 파일 경로: {status_file}")
     print(f"부분 결과 파일 경로: {partial_results_file}")
else:
     print("!! [경고] 결과 저장 경로(result_dir)가 설정되지 않았거나 유효하지 않아 체크포인팅 불가.")

start_index = 0 # 평가 시작할 인덱스 초기화
all_results_list = [] # 이전 결과 누적용 리스트 초기화

if result_dir_valid: # 저장 경로가 유효할 때만 체크포인트 로드 시도
    # 상태 파일 로드 시도
    if os.path.exists(status_file):
        try:
            with open(status_file, 'r', encoding='utf-8') as f:
                status_data = json.load(f)
                last_processed_index = status_data.get('last_processed_index', -1)
                # 유효한 인덱스이고 데이터셋 크기보다 작을 때만 재개 인덱스 설정
                if isinstance(last_processed_index, int) and -1 <= last_processed_index < len(qa_df):
                    start_index = last_processed_index + 1
                    print(f"  [정보] 상태 파일 로드. 마지막 처리 인덱스: {last_processed_index}. {start_index}부터 재개.")
                else:
                     print(f"  [경고] 상태 파일의 인덱스({last_processed_index})가 유효하지 않음. 처음부터 시작.")
                     start_index = 0
        except (json.JSONDecodeError, TypeError, Exception) as e:
            print(f"  [경고] 상태 파일 로드/파싱 오류: {e}. 처음부터 시작.")
            start_index = 0

    # 부분 결과 파일 로드 시도 (재개 시에만 필요)
    if start_index > 0:
        if os.path.exists(partial_results_file):
            try:
                df_partial = pd.read_csv(partial_results_file, encoding='utf-8-sig')
                if not df_partial.empty:
                    # DataFrame을 리스트[딕셔너리] 형태로 변환하여 all_results_list에 저장
                    all_results_list = df_partial.to_dict('records')
                    # 로드된 결과 수와 시작 인덱스가 일치하는지 확인
                    if len(all_results_list) != start_index:
                         print(f"  [경고] 로드된 결과 수({len(all_results_list)})와 상태 파일 인덱스({start_index-1}) 불일치!")
                         print("     -> 이전 결과는 유지하지만, 잠재적 데이터 손실 가능성 있음.")
                         # 이 경우, start_index를 로드된 결과 수에 맞출지, 상태 파일을 우선할지 결정 필요
                         # 여기서는 상태 파일을 우선함 (상태 파일이 마지막 성공 지점을 더 정확히 나타낼 가능성)
                    else:
                         print(f"  [정보] 이전 결과 {len(all_results_list)}개 로드 완료.")
                else:
                     print("  [정보] 부분 결과 파일이 비어 있습니다. 처음부터 시작합니다.")
                     start_index = 0; all_results_list = []
            except Exception as e:
                print(f"  [경고] 부분 결과 파일 로드 오류: {e}. 이전 결과 없이 시작.")
                start_index = 0; all_results_list = []
        else:
             print("  [경고] 상태 파일은 있으나 부분 결과 파일 없음. 처음부터 시작.")
             start_index = 0; all_results_list = []
else:
    print("!! 체크포인팅 비활성화 (저장 경로 문제).")


# --- 12.5: RAGAS 평가 데이터 준비 ---
# (이전 코드와 동일 - 데이터 준비 로직은 체크포인트와 무관)
print("\n--- 12.5: RAGAS 평가 데이터 준비 ---")
eval_data_list = []
if not qa_df.empty and retriever_available:
    print(f"총 {len(qa_df)}개 질문에 대해 Contexts 검색 및 데이터 형식 변환 중...")
    for index, row in tqdm(qa_df.iterrows(), total=qa_df.shape[0], desc="RAGAS 데이터 준비"):
        question_text = row.get(QUESTION_COLUMN)
        if pd.isna(question_text) or not str(question_text).strip(): continue
        question_text = str(question_text).strip()
        answer_text = str(row.get(ANSWER_COLUMN, "")).strip() if answer_column_exists else "N/A"
        ground_truths_input = row.get(GROUND_TRUTHS_COLUMN) if ground_truths_column_exists else None
        retrieved_contexts = []
        ground_truths_list = []
        try:
            if ground_truths_column_exists and ground_truths_input is not None:
                current_gt_input = str(ground_truths_input).strip()
                if current_gt_input:
                    try:
                        if current_gt_input.startswith('[') and current_gt_input.endswith(']'):
                            parsed_list = json.loads(current_gt_input)
                            if isinstance(parsed_list, list):
                                ground_truths_list = [str(item).strip() for item in parsed_list if item is not None and str(item).strip()]
                            else: ground_truths_list = [current_gt_input]
                        else: ground_truths_list = [current_gt_input]
                    except json.JSONDecodeError: ground_truths_list = [current_gt_input]
                    except Exception as json_e: ground_truths_list = [current_gt_input]; print(f"GT 파싱 오류: {json_e}")

            retrieved_docs = retriever.invoke(question_text)
            retrieved_contexts = [doc.page_content for doc in retrieved_docs]
            eval_item = {"question": question_text, "contexts": retrieved_contexts, "ground_truths": ground_truths_list, "ground_truth": answer_text}
            eval_data_list.append(eval_item)
        except Exception as e:
            print(f"\n!! 데이터 준비 중 오류 (질문: '{question_text}'): {e}"); traceback.print_exc()
            eval_data_list.append({"question": question_text, "contexts": [], "ground_truths": [], "ground_truth": ""})
    print(f"[완료] RAGAS 평가 데이터 {len(eval_data_list)}개 준비 완료.")
else:
    if qa_df.empty: print("!! 평가 데이터(qa_df)가 비어있거나 로드되지 않아 데이터 준비 불가.")
    if not retriever_available: print("!! Retriever가 없어 Contexts를 검색할 수 없으므로 데이터 준비 불가.")


# --- 12.6: Hugging Face Dataset 형식으로 변환 ---
print("\n--- 12.6: Hugging Face Dataset 변환 ---")
eval_dataset = None
if eval_data_list and Dataset: # Dataset 클래스 로드 성공 시
    try:
        # eval_data_list의 모든 항목이 RAGAS에서 요구하는 키를 가지고 있는지 확인하는 것이 좋음
        # 예: all(k in item for k in ["question", "contexts", "ground_truths"] for item in eval_data_list)
        eval_dataset = Dataset.from_list(eval_data_list)
        print("[성공] Hugging Face Dataset으로 변환 완료.")
        print(f"   데이터셋 컬럼: {eval_dataset.column_names}")
        # print(f"   데이터셋 미리보기:\n{eval_dataset[:3]}") # 샘플 확인
    except Exception as e:
        print(f"!! [오류] Dataset 변환 중 오류: {e}"); traceback.print_exc(); eval_dataset = None
else: print("!! [정보] 생성된 평가 데이터가 없거나 Dataset 클래스 로드 실패하여 변환 불가.")

# --- 12.7: RAGAS 평가 실행 (배치 처리, 체크포인팅, 지연 시간 추가) ---
print("\n--- 12.7: RAGAS 평가 실행 (배치 처리, 체크포인팅, 지연 시간 추가) ---")
evaluation_df = None # 최종 결과 DataFrame 초기화
eval_batch_size = 10   # ===> 배치 크기 설정 (API 제한 고려, 필요시 1로) <===
# seconds_to_wait_between_batches = 65 # ===> 배치 간 대기 시간 (초) <===(필요하면 실행)
# seconds_to_wait_on_error = 30    # ===> 오류 발생 시 다음 배치 전 대기 시간 (초) <===(필요하면 실행)

# 평가 실행 조건 확인
evaluation_possible = (
    RAGAS_AVAILABLE and
    llm_available and
    retriever_available and # Retriever는 데이터 준비에서 사용되었으므로 재확인
    eval_dataset and        # 데이터셋 변환 성공 여부
    evaluate and context_precision and context_recall and
    result_dir_valid      # 체크포인팅 경로 유효성 확인
)

if evaluation_possible:
    # 사용할 평가 지표 정의
    metrics_to_evaluate = []
    if context_precision: metrics_to_evaluate.append(context_precision)
    else: print("[경고] context_precision 메트릭 사용 불가.")
    if context_recall and ground_truths_column_exists: metrics_to_evaluate.append(context_recall)
    elif context_recall: print("[경고] context_recall 메트릭 사용 불가 (ground_truths 컬럼 없음).")
    else: print("[경고] context_recall 메트릭 자체 사용 불가.")

    if not metrics_to_evaluate:
        print("!! [오류] 평가할 유효한 RAGAS 메트릭이 없습니다.")
    else:
        print(f"평가 지표: {[m.name for m in metrics_to_evaluate]}")
        # start_index 유효성 재확인 (데이터셋 크기보다 크면 안됨)
        if start_index >= len(eval_dataset):
            print(f"[정보] start_index({start_index})가 데이터셋 크기({len(eval_dataset)})보다 크거나 같으므로, 모든 평가가 이미 완료되었습니다.")
            # 이 경우, 이전 결과를 로드했으므로 아래 로직은 실행되지 않음
        else:
            print(f"총 {len(eval_dataset)}개 중 {start_index}부터 {eval_batch_size}개씩 배치 처리...")
            overall_progress = tqdm(total=len(eval_dataset), initial=start_index, desc="전체 평가 진행률")

            # start_index부터 끝까지 eval_batch_size 간격으로 반복
            for i in range(start_index, len(eval_dataset), eval_batch_size):
                batch_end_index = min(i + eval_batch_size, len(eval_dataset))
                print(f"\n--- 배치 처리 시작: 인덱스 {i} ~ {batch_end_index - 1} ---")
                # Dataset 객체는 슬라이싱 대신 select 사용
                batch_dataset = eval_dataset.select(range(i, batch_end_index))

                if len(batch_dataset) == 0:
                     print("  [정보] 현재 배치가 비어있어 건너<0xEB><0x9B><0x81>니다.")
                     continue

                batch_success = False # 현재 배치 성공 여부 플래그
                try:
                    # RAGAS 평가 실행
                    print(f"  RAGAS 평가 실행 (배치 크기: {len(batch_dataset)})...")
                    eval_start_time = time.time()
                    batch_result = evaluate(
                        dataset=batch_dataset,
                        metrics=metrics_to_evaluate,
                        llm=ragas_llm,
                        embeddings=ragas_embeddings if embeddings_available else None,
                        raise_exceptions=False # 오류 시 NaN 반환 유지
                    )
                    eval_end_time = time.time()
                    print(f"  RAGAS evaluate() 함수 실행 시간: {eval_end_time - eval_start_time:.2f}초")

                    # 결과 처리 및 저장
                    if batch_result:
                        batch_result_df = batch_result.to_pandas()
                        print(f"  배치 평가 완료. 결과 {len(batch_result_df)}개 행 생성.")

                        # 결과 누적
                        all_results_list.extend(batch_result_df.to_dict('records'))

                        # 부분 결과를 CSV에 추가 (헤더는 파일 없을 때만 쓰기)
                        is_first_write = not os.path.exists(partial_results_file) if partial_results_file else True
                        if partial_results_file:
                            try:
                                batch_result_df.to_csv(partial_results_file, mode='a', header=is_first_write, index=False, encoding='utf-8-sig')
                                print(f"  부분 결과를 '{os.path.basename(partial_results_file)}'에 추가 저장 완료.")
                            except Exception as csv_e: print(f"!! 부분 결과 CSV 저장 오류: {csv_e}")

                        # 상태 파일 업데이트 (현재 배치의 마지막 인덱스로)
                        last_processed_in_batch = batch_end_index - 1
                        if status_file:
                             try:
                                 with open(status_file, 'w', encoding='utf-8') as f: json.dump({'last_processed_index': last_processed_in_batch}, f)
                                 print(f"  상태 파일 업데이트 완료 (마지막 처리 인덱스: {last_processed_in_batch}).")
                             except Exception as status_save_e: print(f"!! 상태 파일 저장 중 오류: {status_save_e}")

                        # tqdm 진행률 업데이트
                        overall_progress.update(len(batch_dataset))
                        batch_success = True # 배치 성공

                    else:
                        print("!! 배치 평가 결과가 비어있습니다.")
                        overall_progress.update(len(batch_dataset)) # 결과 없어도 진행률 업데이트

                except Exception as e:
                    print(f"!! [오류] 배치 {i}~{batch_end_index-1} RAGAS 평가 실행 중 예외 발생: {e}")
                    print("   다음 배치로 넘어갑니다. API 할당량, 네트워크, 데이터 형식 등을 확인하세요.")
                    traceback.print_exc()
                    # 오류 발생 시에도 진행 상황은 기록 (오류 난 배치는 결과에 포함 안 됨)
                    last_processed_before_error = i - 1 # 오류 난 배치의 시작 이전 인덱스
                    if last_processed_before_error >= 0 and status_file:
                        try:
                            with open(status_file, 'w', encoding='utf-8') as f: json.dump({'last_processed_index': last_processed_before_error}, f)
                            print(f"  (오류 발생 전 마지막 처리 인덱스: {last_processed_before_error}로 상태 저장)")
                        except Exception as status_save_e: print(f"!! 상태 파일 저장 중 오류: {status_save_e}")
                    overall_progress.update(len(batch_dataset)) # 오류난 배치도 진행률 업데이트

                # ===> 다음 배치를 위해 대기 (정상/오류 상황 구분) <===(배치 관련 코드 - 필요하면 실행)
                # # 마지막 배치가 아닐 경우에만 대기
                # if batch_end_index < len(eval_dataset):
                #     wait_time = seconds_to_wait_between_batches if batch_success else seconds_to_wait_on_error
                #     wait_reason = "정상 처리 완료" if batch_success else "오류 발생"
                #     print(f"\n  {wait_reason}. 다음 배치를 위해 {wait_time}초 대기...")
                #     time.sleep(wait_time)

            # 루프 완료 후 tqdm 닫기
            overall_progress.close()

            # 최종 DataFrame 생성
            if all_results_list:
                try:
                    evaluation_df = pd.DataFrame(all_results_list)
                    print(f"\n[성공] 전체 {len(evaluation_df)}개 항목에 대한 RAGAS 평가 결과 통합 완료.")
                    # 최종 결과 파일 저장 (선택 사항)
                    if result_dir_valid:
                        final_results_file = os.path.join(result_dir, "ragas_eval_retriever_results_final.csv")
                        try:
                            evaluation_df.to_csv(final_results_file, index=False, encoding='utf-8-sig')
                            print(f"  최종 결과가 '{os.path.basename(final_results_file)}'에 저장되었습니다.")
                        except Exception as final_csv_e:
                             print(f"!! 최종 결과 CSV 저장 오류: {final_csv_e}")
                except Exception as df_e:
                    print(f"!! 최종 DataFrame 생성 중 오류: {df_e}")
            else:
                print("\n[정보] 처리된 결과가 없습니다.")

else:
    # 평가 실행 불가 사유 요약
    missing_components = []
    if not RAGAS_AVAILABLE: missing_components.append("RAGAS 라이브러리")
    if not llm_available: missing_components.append("LLM (또는 RAGAS 래퍼)")
    if not retriever_available: missing_components.append("Retriever") # 추가
    if not eval_dataset: missing_components.append("평가 데이터셋")
    if not (context_precision or context_recall): missing_components.append("RAGAS 메트릭")
    if not result_dir_valid: missing_components.append("유효한 결과 저장 경로(result_dir)")
    print(f"!! {', '.join(missing_components)} 준비 안 됨. RAGAS 평가 불가.")

# --- 12.8: 전체 평가 결과 요약 ---
# (출력 방식 수정: 전체 결과 표시)
print("\n--- 12.8: RAGAS Retriever 전체 평가 결과 요약 ---")
if evaluation_df is not None and not evaluation_df.empty:
    metrics_to_summarize = []
    if context_precision and 'context_precision' in evaluation_df.columns: metrics_to_summarize.append('context_precision')
    if context_recall and 'context_recall' in evaluation_df.columns: metrics_to_summarize.append('context_recall')

    if metrics_to_summarize:
        # === 수정된 부분: 전체 결과 출력 ===
        print("\n=== 전체 평가 결과 ===") # <<< 헤더 텍스트 변경
        with pd.option_context('display.max_rows', None, # 모든 행 출력
                               'display.max_columns', None, # 모든 열 출력
                               'display.width', 1000):       # 너비 설정 (필요시 조정)
             # <<< .head() 제거하여 전체 DataFrame 출력 >>>
             print(evaluation_df[metrics_to_summarize].round(4).to_string(na_rep='NaN'))
        # ==================================

        print("\n=== 지표별 통계 ===") # <<< 이 부분은 그대로 유지
        stats = evaluation_df[metrics_to_summarize].describe().round(4)
        print(stats)

        nan_counts = evaluation_df[metrics_to_summarize].isna().sum() # <<< 이 부분은 그대로 유지
        if nan_counts.sum() > 0:
             print("\n[참고] NaN 발생 현황:")
             print(nan_counts[nan_counts > 0])
        else: print("\n[정보] 모든 유효 항목에서 점수가 계산되었습니다.")

    else: print("\n!! evaluation_df에 요약할 유효한 메트릭 컬럼이 없습니다.")

    print(f"\n**참고:**") # <<< 이 부분은 그대로 유지
    print(f"  - 'context_recall': Google Sheet '{GROUND_TRUTHS_COLUMN}' 컬럼 기준.")
    print(f"  - 'context_precision': '{QUESTION_COLUMN}', '{ANSWER_COLUMN}'(있을 경우) 컬럼 참조 가능.")
else: print("RAGAS 평가가 실행되지 않았거나 오류가 발생하여 요약할 결과가 없습니다.")

# <<< 마지막 print문의 설명도 현재 상태에 맞게 유지 또는 수정 >>>
print("\n--- 단계 12: Retriever 성능 평가 (RAGAS) 완료 (체크포인팅 및 시간 지연 적용) ---")

# # --- 12.8: 전체 평가 결과 요약 ---(상위 5개 샘플만 확인할 때)
# # (이전 코드와 거의 동일 - 변수명/로그 약간 수정)
# print("\n--- 12.8: RAGAS Retriever 전체 평가 결과 요약 ---")
# if evaluation_df is not None and not evaluation_df.empty:
#     metrics_to_summarize = []
#     if context_precision and 'context_precision' in evaluation_df.columns: metrics_to_summarize.append('context_precision')
#     if context_recall and 'context_recall' in evaluation_df.columns: metrics_to_summarize.append('context_recall')

#     if metrics_to_summarize:
#         print("\n=== 전체 결과 (상위 5개 샘플) ===")
#         with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
#              print(evaluation_df[metrics_to_summarize].round(4).head().to_string(na_rep='NaN'))

#         print("\n=== 지표별 통계 ===")
#         stats = evaluation_df[metrics_to_summarize].describe().round(4)
#         print(stats)

#         nan_counts = evaluation_df[metrics_to_summarize].isna().sum()
#         if nan_counts.sum() > 0:
#              print("\n[참고] NaN 발생 현황:")
#              print(nan_counts[nan_counts > 0])
#         else: print("\n[정보] 모든 유효 항목에서 점수가 계산되었습니다.")

#     else: print("\n!! evaluation_df에 요약할 유효한 메트릭 컬럼이 없습니다.")

#     print(f"\n**참고:**")
#     print(f"  - 'context_recall': Google Sheet '{GROUND_TRUTHS_COLUMN}' 컬럼 기준.")
#     print(f"  - 'context_precision': '{QUESTION_COLUMN}', '{ANSWER_COLUMN}'(있을 경우) 컬럼 참조 가능.")
# else: print("RAGAS 평가가 실행되지 않았거나 오류가 발생하여 요약할 결과가 없습니다.")

# print("\n--- 단계 12: Retriever 성능 평가 (RAGAS) 완료 (체크포인팅 및 시간 지연 적용) ---")


--- 단계 12: RAGAS Retriever 평가 시작 (체크포인팅 및 지연 시간 적용) ---
[정보] Google Colab 환경 감지됨.
[정보] RAGAS 및 관련 라이브러리 임포트 확인.

--- 12.1: Google 인증 및 gspread 클라이언트 확인 ---
[성공] Google 인증 및 gspread 클라이언트 준비 완료.

--- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---
[성공] RAGAS LLM 래퍼 초기화 (모델: gpt-4.1-nano)
[성공] RAGAS Embedding 래퍼 초기화 완료.

--- 12.3: 평가 데이터셋 로드 ---
Google Sheet 'filtered_qa_dataset' 로드 시도...
  워크시트 데이터 로드 중...
  [성공] Google Sheet 로드 완료: 89개 행
     컬럼 목록: ['number', 'question', 'answer', 'ground_truths']
  [정보] 'ground_truths' 컬럼 확인.
  [정보] 'answer' 컬럼 확인.

--- 12.4: 체크포인트 설정 및 로드 ---
결과 저장 경로 확인: /content/drive/MyDrive/nomu_rag_result
상태 파일 경로: /content/drive/MyDrive/nomu_rag_result/ragas_eval_retriever_status.json
부분 결과 파일 경로: /content/drive/MyDrive/nomu_rag_result/ragas_eval_retriever_results_partial.csv

--- 12.5: RAGAS 평가 데이터 준비 ---
총 89개 질문에 대해 Contexts 검색 및 데이터 형식 변환 중...


RAGAS 데이터 준비:   0%|          | 0/89 [00:00<?, ?it/s]

[완료] RAGAS 평가 데이터 89개 준비 완료.

--- 12.6: Hugging Face Dataset 변환 ---
[성공] Hugging Face Dataset으로 변환 완료.
   데이터셋 컬럼: ['question', 'contexts', 'ground_truths', 'ground_truth']

--- 12.7: RAGAS 평가 실행 (배치 처리, 체크포인팅, 지연 시간 추가) ---
평가 지표: ['context_precision', 'context_recall']
총 89개 중 0부터 10개씩 배치 처리...


전체 평가 진행률:   0%|          | 0/89 [00:00<?, ?it/s]


--- 배치 처리 시작: 인덱스 0 ~ 9 ---
  RAGAS 평가 실행 (배치 크기: 10)...


...

  RAGAS evaluate() 함수 실행 시간: 16.38초
  배치 평가 완료. 결과 10개 행 생성.
  부분 결과를 'ragas_eval_retriever_results_partial.csv'에 추가 저장 완료.
  상태 파일 업데이트 완료 (마지막 처리 인덱스: 69).

--- 배치 처리 시작: 인덱스 70 ~ 79 ---
  RAGAS 평가 실행 (배치 크기: 10)...


Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

  RAGAS evaluate() 함수 실행 시간: 13.94초
  배치 평가 완료. 결과 10개 행 생성.
  부분 결과를 'ragas_eval_retriever_results_partial.csv'에 추가 저장 완료.
  상태 파일 업데이트 완료 (마지막 처리 인덱스: 79).

--- 배치 처리 시작: 인덱스 80 ~ 88 ---
  RAGAS 평가 실행 (배치 크기: 9)...


Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

  RAGAS evaluate() 함수 실행 시간: 11.78초
  배치 평가 완료. 결과 9개 행 생성.
  부분 결과를 'ragas_eval_retriever_results_partial.csv'에 추가 저장 완료.
  상태 파일 업데이트 완료 (마지막 처리 인덱스: 88).

[성공] 전체 89개 항목에 대한 RAGAS 평가 결과 통합 완료.
  최종 결과가 'ragas_eval_retriever_results_final.csv'에 저장되었습니다.

--- 12.8: RAGAS Retriever 전체 평가 결과 요약 ---

=== 전체 평가 결과 ===
    context_precision  context_recall
0              1.0000          1.0000
1              1.0000          1.0000
2              0.8750          0.8966
3              0.5909          1.0000
4              0.8667          1.0000
5              0.8583          1.0000
6              0.7611          0.0000
7              0.8917          1.0000
8              0.9889          1.0000
9              0.7679          1.0000
10             1.0000          0.5000
11             0.6667          0.5000
12             0.7917          1.0000
13             1.0000          1.0000
14             1.0000          0.6667
15             0.7500          1.0000
16             0.7917          1.0000

# (옵션)12.9: 저장된 RAGAS 평가 결과 전체 보기

In [ ]:
# === 단계 12.9: 저장된 RAGAS 평가 결과 전체 보기 ===

import pandas as pd
import os
import numpy as np # NaN 값 처리 위해 임포트

print("\n--- 단계 12.9: 저장된 RAGAS 평가 결과 로드 및 전체 보기 ---")

# --- 결과 파일 경로 확인 ---
# 이전 단계에서 정의된 result_dir 변수가 필요합니다.
# 그리고 저장된 파일 이름 (final 또는 partial)을 확인해야 합니다.
result_dir_valid = False
evaluation_df = None # 결과를 담을 DataFrame 초기화
results_file_path = None # 실제로 로드할 파일 경로

if 'result_dir' in locals() and isinstance(result_dir, str) and os.path.isdir(result_dir):
    print(f"결과 저장 경로 확인: {result_dir}")
    result_dir_valid = True
    # 최종 결과 파일과 부분 결과 파일 경로 정의
    final_results_file = os.path.join(result_dir, "ragas_eval_retriever_results_final.csv")
    partial_results_file = os.path.join(result_dir, "ragas_eval_retriever_results_partial.csv")

    # 최종 결과 파일이 있으면 그것을 우선 로드, 없으면 부분 결과 파일 로드 시도
    if os.path.exists(final_results_file):
        results_file_path = final_results_file
        print(f"최종 결과 파일 로드 시도: {os.path.basename(results_file_path)}")
    elif os.path.exists(partial_results_file):
        results_file_path = partial_results_file
        print(f"부분 결과 파일 로드 시도: {os.path.basename(results_file_path)}")
    else:
        print(f"!! [오류] 결과 디렉토리({result_dir})에서 최종 또는 부분 결과 CSV 파일을 찾을 수 없습니다.")

else:
    print("!! [경고] 결과 저장 경로(result_dir)가 설정되지 않았거나 유효하지 않습니다.")

# --- 결과 파일 로드 및 처리 ---
if results_file_path:
    try:
        evaluation_df = pd.read_csv(results_file_path, encoding='utf-8-sig') # BOM 문제 방지 위해 utf-8-sig 사용
        print(f"[성공] '{os.path.basename(results_file_path)}' 파일 로드 완료. 총 {len(evaluation_df)}개 행.")

        # DataFrame이 비어있는지 확인
        if evaluation_df.empty:
            print("!! [정보] 로드된 평가 결과 파일이 비어 있습니다.")
            evaluation_df = None # 비어 있으면 None 처리

    except FileNotFoundError:
        print(f"!! [오류] 파일을 찾을 수 없습니다: {results_file_path}")
        evaluation_df = None
    except Exception as e:
        print(f"!! [오류] CSV 파일 로드 중 오류 발생: {e}")
        evaluation_df = None

# --- 로드된 결과 출력 및 요약 (단계 12.8 코드 재사용 및 수정) ---
if evaluation_df is not None and not evaluation_df.empty:
    # 출력할 메트릭 컬럼 확인
    metrics_to_summarize = []
    # 필요한 메트릭 임포트 여부도 확인 (이전 셀에서 로드 실패했을 수 있음)
    context_precision_available = 'context_precision' in locals() and context_precision is not None
    context_recall_available = 'context_recall' in locals() and context_recall is not None

    if context_precision_available and 'context_precision' in evaluation_df.columns: metrics_to_summarize.append('context_precision')
    if context_recall_available and 'context_recall' in evaluation_df.columns: metrics_to_summarize.append('context_recall')

    if metrics_to_summarize:
        print(f"\n=== 전체 평가 결과 ({', '.join(metrics_to_summarize)}) ===")
        with pd.option_context('display.max_rows', None, # 모든 행 출력
                               'display.max_columns', None, # 모든 열 출력
                               'display.width', 1000):       # 너비 설정
             # 전체 DataFrame의 해당 컬럼 출력
             print(evaluation_df[metrics_to_summarize].round(4).to_string(na_rep='NaN'))

        print("\n=== 지표별 통계 ===")
        stats = evaluation_df[metrics_to_summarize].describe().round(4)
        print(stats)

        nan_counts = evaluation_df[metrics_to_summarize].isna().sum()
        if nan_counts.sum() > 0:
             print("\n[참고] NaN 발생 현황:")
             print(nan_counts[nan_counts > 0])
        else: print("\n[정보] 모든 유효 항목에서 점수가 계산되었습니다.")

    else:
        print("\n!! 로드된 DataFrame에 요약할 유효한 RAGAS 메트릭 컬럼('context_precision', 'context_recall')이 없습니다.")
        print(f"   사용 가능한 컬럼: {evaluation_df.columns.tolist()}")

    # 참고 정보는 필요하면 유지
    # print(f"\n**참고:**")
    # print(f"  - 'context_recall': Google Sheet '{GROUND_TRUTHS_COLUMN}' 컬럼 기준.")
    # print(f"  - 'context_precision': '{QUESTION_COLUMN}', '{ANSWER_COLUMN}'(있을 경우) 컬럼 참조 가능.")

else:
    print("\n저장된 평가 결과를 로드하지 못했거나 데이터가 비어있어 결과를 표시할 수 없습니다.")

print("\n--- 단계 12.9: 저장된 RAGAS 평가 결과 보기 완료 ---")


--- 단계 12.9: 저장된 RAGAS 평가 결과 로드 및 전체 보기 ---
결과 저장 경로 확인: /content/drive/MyDrive/nomu_rag_result
최종 결과 파일 로드 시도: ragas_eval_retriever_results_final.csv
[성공] 'ragas_eval_retriever_results_final.csv' 파일 로드 완료. 총 89개 행.

=== 전체 평가 결과 (context_precision, context_recall) ===
    context_precision  context_recall
0              0.6667          1.0000
1              1.0000          1.0000
2              0.6429          0.8974
3              0.6984          1.0000
4              0.7500          1.0000
5              0.7052          1.0000
6              0.8333          1.0000
7              0.5873          1.0000
8              0.5694          1.0000
9              0.6429          1.0000
10             0.7500          1.0000
11             0.8056          1.0000
12             0.7254          1.0000
13             0.8774          1.0000
14             0.7962          1.0000
15             1.0000          1.0000
16             0.6667          1.0000
17             0.7454          0.0000
18     

# (옵션)*****체크포인트는 없지만 에러없이 잘 실행되는 RAGAS Retriever 평가 코드임.

In [ ]:
# -*- coding: utf-8 -*-
# === 단계 12: RAGAS Retriever 평가 (filtered_qa_dataset 사용 + 시간 지연 추가) ===

# 기능:
# 1. Google Sheet에서 평가 데이터셋(question, answer, ground_truths) 로드
# 2. 각 질문에 대해 주어진 Retriever를 실행하여 Contexts 검색
# 3. RAGAS 라이브러리를 사용하여 Context Precision 및 Context Recall 평가 (배치 처리 및 지연 시간 적용)
# 4. 개별 질문별 점수 및 전체 평균 점수 출력

# --- 12.0: 필요 라이브러리 임포트 및 재확인 ---
print("\n--- 단계 12: RAGAS Retriever 평가 시작 (시간 지연 적용) ---") # 로그 수정
import os
import pandas as pd
import gspread
# google.colab 라이브러리는 Colab 환경에서만 사용 가능합니다.
# 로컬 환경에서는 다른 인증 방식을 사용해야 할 수 있습니다.
try:
    from google.colab import auth
    print("[정보] Google Colab 환경 감지됨.")
    COLAB_ENV = True
except ImportError:
    print("[정보] Google Colab 환경 아님. 로컬 인증 필요.")
    COLAB_ENV = False
from google.auth import default as google_auth_default
import time
from tqdm.notebook import tqdm # Colab/Jupyter 환경용 진행 바
import numpy as np
import json
import warnings
import traceback # 상세 오류 출력

# RAGAS 및 관련 라이브러리 임포트 (오류 발생 시 메시지 출력)
try:
    from datasets import Dataset
    from ragas import evaluate
    from ragas.metrics import context_precision, context_recall
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper
    print("[정보] RAGAS 및 관련 라이브러리 임포트 확인.")
    RAGAS_AVAILABLE = True
except ImportError as e:
    print(f"!! [오류] RAGAS 관련 라이브러리 임포트 실패: {e}")
    Dataset = None; evaluate = None; context_precision = None; context_recall = None;
    LangchainLLMWrapper = None; LangchainEmbeddingsWrapper = None
    RAGAS_AVAILABLE = False

warnings.filterwarnings("ignore") # 경고 메시지 숨기기 (선택 사항)

# --- 12.1: Google 인증 및 gspread 클라이언트 확인 ---
print("\n--- 12.1: Google 인증 및 gspread 클라이언트 확인 ---")
gc = None # gspread 클라이언트 초기화
try:
    if COLAB_ENV:
        auth.authenticate_user() # Colab 사용자 인증
    creds, _ = google_auth_default() # 기본 인증 정보 가져오기
    gc = gspread.authorize(creds)
    print("[성공] Google 인증 및 gspread 클라이언트 준비 완료.")
except Exception as e:
    print(f"!! [실패] Google 인증 또는 gspread 클라이언트 생성 오류: {e}.")

# --- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---
print("\n--- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---")

# 이전에 정의된 llm, retriever, embeddings 객체가 있다고 가정합니다.
llm_available = 'llm' in locals() and llm is not None
retriever_available = 'retriever' in locals() and retriever is not None
embeddings_available = 'embeddings' in locals() and embeddings is not None
ragas_llm = None
ragas_embeddings = None

if RAGAS_AVAILABLE: # RAGAS 라이브러리가 성공적으로 임포트되었을 때만
    if llm_available:
        if LangchainLLMWrapper:
            try:
                model_name_display = getattr(llm, 'model', getattr(llm, 'model_name', 'N/A'))
                ragas_llm = LangchainLLMWrapper(llm) # 키워드 인자 없이 객체만 전달
                print(f"[성공] RAGAS LLM 래퍼 초기화 완료 (모델: {model_name_display})")
            except Exception as e:
                print(f"!! [오류] RAGAS LLM 래퍼 초기화 실패: {e}"); llm_available = False
    else: print("!! [오류] LLM 객체('llm')가 정의되지 않았습니다.")

    if embeddings_available:
        if LangchainEmbeddingsWrapper:
            try:
                ragas_embeddings = LangchainEmbeddingsWrapper(embeddings) # 키워드 인자 없이 객체만 전달
                print("[성공] RAGAS Embedding 래퍼 초기화 완료.")
            except Exception as e:
                print(f"!! [경고] RAGAS Embedding 래퍼 초기화 오류: {e}."); embeddings_available = False
    else: print("[경고] Embedding 객체('embeddings') 없음. 일부 메트릭 사용 불가.")
else:
    print("!! RAGAS 라이브러리 임포트 실패. RAGAS 평가 불가.")

# --- 12.3: 평가 데이터셋 로드 (Google Sheet: filtered_qa_dataset) ---
# (이전 코드와 동일 - 변경 없음)
print("\n--- 12.3: 평가 데이터셋 로드 ---")
GSHEET_FILE_NAME = "filtered_qa_dataset"
QUESTION_COLUMN = "question"; ANSWER_COLUMN = "answer"; GROUND_TRUTHS_COLUMN = "ground_truths"
qa_df = pd.DataFrame(); ground_truths_column_exists = False; answer_column_exists = False
if gc:
    print(f"Google Sheet '{GSHEET_FILE_NAME}' 로드 시도...")
    try:
        spreadsheet = gc.open(GSHEET_FILE_NAME); worksheet = spreadsheet.get_worksheet(0)
        print("  워크시트 로드 중..."); data = worksheet.get_all_values()
        if len(data) > 1:
            qa_df = pd.DataFrame(data[1:], columns=data[0])
            print(f"  [성공] Google Sheet 로드 완료: {len(qa_df)}개 행"); print(f"     컬럼 목록: {qa_df.columns.tolist()}")
            if QUESTION_COLUMN not in qa_df.columns: raise ValueError(f"'{QUESTION_COLUMN}' 컬럼이 시트에 없습니다.")
            if GROUND_TRUTHS_COLUMN in qa_df.columns: ground_truths_column_exists = True
            else: print(f"  !! [경고] '{GROUND_TRUTHS_COLUMN}' 컬럼 없음. context_recall 평가 결과는 0이 됩니다.")
            if ANSWER_COLUMN in qa_df.columns: answer_column_exists = True
            else: print(f"  !! [경고] '{ANSWER_COLUMN}' 컬럼 없음. context_precision 등 일부 메트릭 결과에 영향 가능.")
        else: print("  !! [오류] Google Sheet에 헤더 외 데이터가 없습니다."); qa_df = pd.DataFrame()
    except gspread.exceptions.SpreadsheetNotFound: print(f"  !! [오류] Google Sheet '{GSHEET_FILE_NAME}'을(를) 찾을 수 없습니다."); qa_df = pd.DataFrame()
    except Exception as e: print(f"  !! [오류] Google Sheet 로드 중 예상치 못한 오류: {e}"); traceback.print_exc(); qa_df = pd.DataFrame()
else: print("!! Google Sheet 클라이언트(gc)가 준비되지 않아 데이터셋을 로드할 수 없습니다.")

# --- 12.4: RAGAS 평가 데이터 준비 ---
# (이전 코드와 동일 - 변경 없음, Ground Truths 파싱 로직 포함)
print("\n--- 12.4: RAGAS 평가 데이터 준비 ---")
eval_data_list = []
if not qa_df.empty and retriever_available:
    print(f"총 {len(qa_df)}개 질문에 대해 Contexts 검색 및 데이터 형식 변환 중...")
    for index, row in tqdm(qa_df.iterrows(), total=qa_df.shape[0], desc="RAGAS 데이터 준비"):
        question_text = row.get(QUESTION_COLUMN)
        if pd.isna(question_text) or not str(question_text).strip(): continue
        question_text = str(question_text).strip()
        answer_text = str(row.get(ANSWER_COLUMN, "")).strip() if answer_column_exists else ""
        ground_truths_input = row.get(GROUND_TRUTHS_COLUMN, "[]") if ground_truths_column_exists else "[]"
        retrieved_contexts = []
        ground_truths_list = []
        try:
            if ground_truths_column_exists and isinstance(ground_truths_input, str) and ground_truths_input.strip():
                try:
                    parsed_list = json.loads(ground_truths_input)
                    if isinstance(parsed_list, list):
                        ground_truths_list = [str(item).strip() for item in parsed_list if item is not None and str(item).strip()]
                    else: ground_truths_list = [ground_truths_input.strip()]
                except (json.JSONDecodeError, TypeError): ground_truths_list = [ground_truths_input.strip()]
            elif isinstance(ground_truths_input, list): ground_truths_list = [str(item).strip() for item in ground_truths_input if item is not None and str(item).strip()]

            retrieved_docs = retriever.invoke(question_text)
            retrieved_contexts = [doc.page_content for doc in retrieved_docs]
            eval_item = {"question": question_text, "contexts": retrieved_contexts, "ground_truths": ground_truths_list, "ground_truth": answer_text}
            eval_data_list.append(eval_item)
        except Exception as e:
            print(f"\n!! 데이터 준비 중 오류 (질문: '{question_text}'): {e}")
            traceback.print_exc()
            eval_data_list.append({"question": question_text, "contexts": [], "ground_truths": [], "ground_truth": ""})
    print(f"[완료] RAGAS 평가 데이터 {len(eval_data_list)}개 준비 완료.")
else:
    if qa_df.empty: print("!! 평가 데이터(qa_df)가 비어있거나 로드되지 않아 데이터 준비 불가.")
    if not retriever_available: print("!! Retriever가 없어 Contexts를 검색할 수 없으므로 데이터 준비 불가.")

# --- 12.5: Hugging Face Dataset 형식으로 변환 ---
# (이전 코드와 동일 - 변경 없음)
print("\n--- 12.5: Hugging Face Dataset 변환 ---")
eval_dataset = None
if eval_data_list and Dataset: # Dataset 클래스 로드 성공 시
    try:
        eval_dataset = Dataset.from_list(eval_data_list)
        print("[성공] Hugging Face Dataset으로 변환 완료.")
        print(f"   데이터셋 컬럼: {eval_dataset.column_names}")
        # 샘플 데이터 확인 (선택 사항)
        # num_samples_to_show = min(3, len(eval_dataset))
        # print(f"\n   데이터셋 샘플 확인 (처음 {num_samples_to_show}개):")
        # for i in range(num_samples_to_show): print(f"   --- 항목 #{i} ---\n{eval_dataset[i]}\n")
    except Exception as e:
        print(f"!! [오류] Dataset 변환 중 오류: {e}"); traceback.print_exc(); eval_dataset = None
else: print("!! [정보] 생성된 평가 데이터가 없거나 Dataset 클래스 로드 실패하여 변환 불가.")

# --- 12.6: RAGAS 평가 실행 (배치 처리 및 시간 지연 추가) ---
print("\n--- 12.6: RAGAS 평가 실행 (배치 처리 및 시간 지연 추가) ---")
evaluation_df = None # 평가 결과 저장 DataFrame 초기화
all_results_list = [] # 배치 결과 누적용 리스트
eval_batch_size = 3   # ===> 배치 크기 설정 (API 제한 고려, 필요시 1로) <===
seconds_to_wait_between_batches = 30 # ===> 배치 간 대기 시간 (초) <===
seconds_to_wait_on_error = 30    # ===> 오류 발생 시 다음 배치 전 대기 시간 (초) <===

# 평가 실행 조건 확인
evaluation_possible = (
    RAGAS_AVAILABLE and # RAGAS 라이브러리 로드 성공 여부
    llm_available and   # LLM 래퍼 초기화 성공 여부
    eval_dataset and    # 데이터셋 변환 성공 여부
    evaluate and context_precision and context_recall # Ragas 함수/메트릭 확인
)

if evaluation_possible:
    # 사용할 평가 지표 정의
    metrics_to_evaluate = []
    if context_precision: metrics_to_evaluate.append(context_precision)
    else: print("[경고] context_precision 메트릭 사용 불가.")
    if context_recall and ground_truths_column_exists: metrics_to_evaluate.append(context_recall)
    elif context_recall: print("[경고] context_recall 메트릭 사용 불가 (ground_truths 컬럼 없음).")
    else: print("[경고] context_recall 메트릭 자체 사용 불가.")

    if not metrics_to_evaluate:
        print("!! [오류] 평가할 유효한 RAGAS 메트릭이 없습니다.")
    else:
        print(f"평가 지표: {[m.name for m in metrics_to_evaluate]}")
        print(f"총 {len(eval_dataset)}개 항목을 {eval_batch_size}개씩 배치 처리...")

        # tqdm으로 전체 진행률 표시 (데이터셋 크기 기준)
        # 여기서는 배치 루프 내에서 업데이트하는 대신 전체 루프 기준으로 생성
        overall_progress = tqdm(total=len(eval_dataset), desc="전체 평가 진행률")

        # 전체 데이터셋을 배치 크기만큼 나누어 처리
        for i in range(0, len(eval_dataset), eval_batch_size):
            batch_end_index = min(i + eval_batch_size, len(eval_dataset))
            print(f"\n--- 배치 처리 시작: 인덱스 {i} ~ {batch_end_index - 1} ---")
            batch_dataset = eval_dataset.select(range(i, batch_end_index)) # 현재 배치 슬라이싱

            if len(batch_dataset) == 0:
                 print("  [정보] 현재 배치가 비어있어 건너<0xEB><0x9B><0x81>니다.")
                 continue

            try:
                # RAGAS 평가 실행
                print(f"  RAGAS 평가 실행 (배치 크기: {len(batch_dataset)})...")
                eval_start_time = time.time()
                batch_result = evaluate(
                    dataset=batch_dataset,           # 현재 배치 데이터셋
                    metrics=metrics_to_evaluate,    # 평가 지표 리스트
                    llm=ragas_llm,                  # Ragas LLM 래퍼
                    embeddings=ragas_embeddings if embeddings_available else None,
                    # raise_exceptions=False 기본값 사용 (개별 오류 시 NaN)
                    # batch_size 인자는 evaluate 함수에 직접 없음 (데이터셋 크기로 결정됨)
                )
                eval_end_time = time.time()
                print(f"  RAGAS evaluate() 함수 실행 시간: {eval_end_time - eval_start_time:.2f}초")

                # 결과 처리
                if batch_result:
                    batch_result_df = batch_result.to_pandas()
                    print(f"  배치 평가 완료. 결과 {len(batch_result_df)}개 행 생성.")
                    all_results_list.extend(batch_result_df.to_dict('records')) # 결과 누적
                    overall_progress.update(len(batch_dataset)) # 성공 시 진행률 업데이트
                else:
                    print("!! 배치 평가 결과가 비어있습니다.")
                    # 결과가 없어도 처리한 것으로 간주하고 진행률 업데이트
                    overall_progress.update(len(batch_dataset))

                # ===> 정상 처리 후, 다음 배치를 위해 지정된 시간만큼 대기 <===
                # 마지막 배치가 아닐 경우에만 대기
                if batch_end_index < len(eval_dataset):
                    print(f"\n  정상 처리 완료. API 호출 제한 방지를 위해 {seconds_to_wait_between_batches}초 대기...")
                    time.sleep(seconds_to_wait_between_batches)

            except Exception as e:
                print(f"!! [오류] 배치 {i}~{batch_end_index-1} RAGAS 평가 중 심각한 오류 발생: {e}")
                print("   다음 배치로 넘어갑니다. API 할당량, 네트워크, 데이터 형식 등을 확인하세요.")
                traceback.print_exc()
                # 오류 발생 시에도 해당 배치만큼 진행률 업데이트 (건너뛴 것으로 처리)
                overall_progress.update(len(batch_dataset))
                # ===> 오류 발생 시 다음 배치를 위해 잠시 대기 <===
                # 마지막 배치가 아닐 경우에만 대기
                if batch_end_index < len(eval_dataset):
                     print(f"  오류 발생. 다음 배치를 위해 {seconds_to_wait_on_error}초 대기...")
                     time.sleep(seconds_to_wait_on_error)

        # 루프 완료 후 tqdm 닫기
        overall_progress.close()

        # 최종 DataFrame 생성
        if all_results_list:
            try:
                evaluation_df = pd.DataFrame(all_results_list)
                print(f"\n[성공] 전체 {len(evaluation_df)}개 항목에 대한 RAGAS 평가 결과 통합 완료.")
                 # 최종 결과 파일 저장 (선택 사항, result_dir 변수 필요)
                 # if 'result_dir' in locals() and result_dir and os.path.isdir(result_dir):
                 #     final_results_file = os.path.join(result_dir, "ragas_eval_retriever_results_final.csv")
                 #     evaluation_df.to_csv(final_results_file, index=False, encoding='utf-8-sig')
                 #     print(f"  최종 결과가 '{os.path.basename(final_results_file)}'에 저장되었습니다.")
            except Exception as df_e:
                 print(f"!! 최종 DataFrame 생성 중 오류: {df_e}")
        else:
            print("\n[정보] 처리된 결과가 없습니다.")

else:
    # 평가 실행 불가 사유 요약
    missing_components = []
    if not RAGAS_AVAILABLE: missing_components.append("RAGAS 라이브러리")
    if not llm_available: missing_components.append("LLM (또는 RAGAS 래퍼)")
    if not eval_dataset: missing_components.append("평가 데이터셋")
    if not (context_precision or context_recall): missing_components.append("RAGAS 메트릭")
    print(f"!! {', '.join(missing_components)} 준비 안 됨. RAGAS 평가 불가.")

# --- 12.7: 전체 평가 결과 요약 ---
# (이전 코드와 동일 - 변경 없음)
print("\n--- 12.7: RAGAS Retriever 전체 평가 결과 요약 ---")
if evaluation_df is not None and not evaluation_df.empty:
    existing_metrics = [m.name for m in metrics_to_evaluate if m and m.name in evaluation_df.columns]
    if existing_metrics:
        print("\n=== 전체 결과 (요약) ===")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
             print(evaluation_df[existing_metrics].round(4).to_string(na_rep='NaN'))
        print("\n=== 지표별 통계 ===")
        stats = evaluation_df[existing_metrics].describe().round(4)
        print(stats)
        nan_counts = evaluation_df[existing_metrics].isna().sum()
        if nan_counts.sum() > 0:
             print("\n[참고] NaN 발생 현황:")
             print(nan_counts[nan_counts > 0])
        else: print("\n[정보] 모든 항목에서 유효한 점수가 계산되었습니다.")
    else: print("\n!! evaluation_df에 요약할 유효한 메트릭 컬럼이 없습니다.")
    print(f"\n**참고:**")
    print(f"  - 'context_recall': Google Sheet '{GROUND_TRUTHS_COLUMN}' 컬럼 기준.")
    print(f"  - 'context_precision': '{QUESTION_COLUMN}', '{ANSWER_COLUMN}'(있을 경우) 컬럼 참조 가능.")
else: print("RAGAS 평가가 실행되지 않았거나 오류가 발생하여 요약할 결과가 없습니다.")

print("\n--- 단계 12: Retriever 성능 평가 (RAGAS) 완료 (시간 지연 적용) ---") # 로그 수정


--- 단계 12: RAGAS Retriever 평가 시작 (시간 지연 적용) ---
[정보] Google Colab 환경 감지됨.
[정보] RAGAS 및 관련 라이브러리 임포트 확인.

--- 12.1: Google 인증 및 gspread 클라이언트 확인 ---
[성공] Google 인증 및 gspread 클라이언트 준비 완료.

--- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---
[성공] RAGAS LLM 래퍼 초기화 완료 (모델: models/gemini-1.5-flash)
[성공] RAGAS Embedding 래퍼 초기화 완료.

--- 12.3: 평가 데이터셋 로드 ---
Google Sheet 'filtered_qa_dataset' 로드 시도...
  워크시트 로드 중...
  [성공] Google Sheet 로드 완료: 89개 행
     컬럼 목록: ['question', 'answer', 'ground_truths']

--- 12.4: RAGAS 평가 데이터 준비 ---
총 89개 질문에 대해 Contexts 검색 및 데이터 형식 변환 중...


RAGAS 데이터 준비:   0%|          | 0/89 [00:00<?, ?it/s]

[완료] RAGAS 평가 데이터 89개 준비 완료.

--- 12.5: Hugging Face Dataset 변환 ---
[성공] Hugging Face Dataset으로 변환 완료.
   데이터셋 컬럼: ['question', 'contexts', 'ground_truths', 'ground_truth']

--- 12.6: RAGAS 평가 실행 (배치 처리 및 시간 지연 추가) ---
평가 지표: ['context_precision', 'context_recall']
총 89개 항목을 3개씩 배치 처리...


전체 평가 진행률:   0%|          | 0/89 [00:00<?, ?it/s]


--- 배치 처리 시작: 인덱스 0 ~ 2 ---
  RAGAS 평가 실행 (배치 크기: 3)...


...

}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].
}
, links {
  description: "Learn more a

  RAGAS evaluate() 함수 실행 시간: 180.26초
  배치 평가 완료. 결과 3개 행 생성.

  정상 처리 완료. API 호출 제한 방지를 위해 30초 대기...

--- 배치 처리 시작: 인덱스 87 ~ 88 ---
  RAGAS 평가 실행 (배치 크기: 2)...


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 45
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 45
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 45
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 45
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 43
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 42
}
].
}
, links {
  description: "Learn more a

  RAGAS evaluate() 함수 실행 시간: 180.25초
  배치 평가 완료. 결과 2개 행 생성.

[성공] 전체 89개 항목에 대한 RAGAS 평가 결과 통합 완료.

--- 12.7: RAGAS Retriever 전체 평가 결과 요약 ---

=== 전체 결과 (요약) ===
    context_precision  context_recall
0                 NaN             NaN
1                 NaN             NaN
2                 NaN             NaN
3                 NaN             NaN
4                 NaN             NaN
5                 NaN             NaN
6                 NaN             NaN
7                 NaN             NaN
8                 NaN             NaN
9                 NaN             NaN
10                NaN             NaN
11                NaN             NaN
12                NaN             NaN
13                NaN             NaN
14                NaN             NaN
15                NaN             NaN
16                NaN             NaN
17                NaN             NaN
18                NaN             NaN
19                NaN             NaN
20                NaN             NaN
2

In [ ]:

# === 단계 12: RAGAS Retriever 평가 (filtered_qa_dataset 사용) ===

# 기능:
# 1. Google Sheet에서 평가 데이터셋(question, answer, ground_truths) 로드
# 2. 각 질문에 대해 주어진 Retriever를 실행하여 Contexts 검색
# 3. RAGAS 라이브러리를 사용하여 Context Precision 및 Context Recall 평가
# 4. 개별 질문별 점수 및 전체 평균 점수 출력

# --- 12.0: 필요 라이브러리 임포트 및 재확인 ---
print("\n--- 단계 12: RAGAS Retriever 평가 시작 ---")
import os
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default as google_auth_default
import time
from tqdm.notebook import tqdm
import numpy as np
import json
import warnings
import traceback # 상세 오류 출력

# RAGAS 및 관련 라이브러리 임포트 (오류 발생 시 메시지 출력)
try:
    from datasets import Dataset
    from ragas import evaluate
    from ragas.metrics import context_precision, context_recall
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper
    print("[정보] RAGAS 및 관련 라이브러리 임포트 확인.")
except ImportError as e:
    print(f"!! [오류] RAGAS 관련 라이브러리 임포트 실패: {e}")
    print("   이전 단계에서 `!pip install ragas datasets` 등이 성공했는지 확인하세요.")
    # 평가 진행 불가 시뮬레이션을 위해 None 설정 (실제로는 오류 발생)
    Dataset = None; evaluate = None; context_precision = None; context_recall = None;
    LangchainLLMWrapper = None; LangchainEmbeddingsWrapper = None

warnings.filterwarnings("ignore") # 경고 메시지 숨기기 (선택 사항)

# --- 12.1: Google 인증 및 gspread 클라이언트 확인 ---
print("\n--- 12.1: Google 인증 및 gspread 클라이언트 확인 ---")
gc = None # gspread 클라이언트 초기화
try:
    # Colab 환경에서는 일반적으로 이전 셀에서 인증 완료됨
    creds, _ = google_auth_default()
    gc = gspread.authorize(creds)
    print("[성공] Google 인증 및 gspread 클라이언트 준비 완료.")
except Exception as e:
    print(f"!! [실패] Google 인증 또는 gspread 클라이언트 생성 오류: {e}.")
    print("   이전 단계의 Google 인증 셀이 성공했는지 확인하세요.")

# --- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---
print("\n--- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---")

# 이전 단계에서 생성된 LLM, Retriever, Embeddings 객체 확인
llm_available = 'llm' in locals() and llm is not None
retriever_available = 'retriever' in locals() and retriever is not None
embeddings_available = 'embeddings' in locals() and embeddings is not None

ragas_llm = None
ragas_embeddings = None

# LLM 래퍼 초기화
if llm_available:
    if LangchainLLMWrapper: # RAGAS 임포트 성공 시
        try:
            # <<< 수정: 키워드 인자 'llm=' 제거 >>>
            ragas_llm = LangchainLLMWrapper(llm)
            # -------------------------------------
            print(f"[성공] RAGAS LLM 래퍼 초기화 완료 (모델: {llm.model})")
        except Exception as e:
            print(f"!! [오류] RAGAS LLM 래퍼 초기화 실패: {e}")
            llm_available = False # 래퍼 초기화 실패 시 평가 불가
    else:
        print("!! Ragas LLM 래퍼 클래스 임포트 실패.")
        llm_available = False
else:
    print("!! [오류] LLM 객체('llm')가 정의되지 않았습니다.")

# (이하 Retriever 및 Embedding 래퍼 초기화 로직은 동일)
if retriever_available: print("[정보] Retriever 객체('retriever') 확인 완료.")
else: print("!! [오류] Retriever 객체('retriever') 없음.")

if embeddings_available:
    if LangchainEmbeddingsWrapper:
        try:
            # <<< 수정: 키워드 인자 'embeddings=' 제거 >>>
            ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)
            # -----------------------------------------
            print("[성공] RAGAS Embedding 래퍼 초기화 완료.")
        except Exception as e:
            print(f"!! [경고] RAGAS Embedding 래퍼 초기화 오류: {e}.")
            embeddings_available = False
    else:
        print("!! Ragas Embedding 래퍼 클래스 임포트 실패.")
        embeddings_available = False
else:
    print("[경고] Embedding 객체('embeddings') 없음. 일부 Ragas 메트릭 사용 불가.")

# --- 12.3: 평가 데이터셋 로드 (Google Sheet: filtered_qa_dataset) ---
print("\n--- 12.3: 평가 데이터셋 로드 ---")
# 사용할 Google Sheet 파일 이름
GSHEET_FILE_NAME = "filtered_qa_dataset"
# 시트 내 컬럼 이름 정의 (실제 시트 헤더와 일치해야 함)
QUESTION_COLUMN = "question"
ANSWER_COLUMN = "answer"             # context_precision 등에서 사용될 수 있음
GROUND_TRUTHS_COLUMN = "ground_truths" # context_recall 평가의 핵심

qa_df = pd.DataFrame() # 데이터 로드용 DataFrame 초기화
ground_truths_column_exists = False
answer_column_exists = False

# Google Sheet 클라이언트가 준비되었을 때만 로드 시도
if gc:
    print(f"Google Sheet '{GSHEET_FILE_NAME}' 로드 시도...")
    try:
        spreadsheet = gc.open(GSHEET_FILE_NAME)
        worksheet = spreadsheet.get_worksheet(0) # 첫 번째 시트 사용 가정
        print("  워크시트 로드 중...")
        data = worksheet.get_all_values() # 시트 전체 데이터 가져오기
        if len(data) > 1: # 헤더 포함 최소 2줄 이상
            qa_df = pd.DataFrame(data[1:], columns=data[0]) # DataFrame 생성
            print(f"  [성공] Google Sheet 로드 완료: {len(qa_df)}개 행")
            print(f"     컬럼 목록: {qa_df.columns.tolist()}")

            # 필수 컬럼 존재 확인
            if QUESTION_COLUMN not in qa_df.columns: raise ValueError(f"'{QUESTION_COLUMN}' 컬럼이 시트에 없습니다.")
            # 선택 컬럼 존재 확인 및 플래그 설정
            if GROUND_TRUTHS_COLUMN in qa_df.columns: ground_truths_column_exists = True
            else: print(f"  !! [경고] '{GROUND_TRUTHS_COLUMN}' 컬럼 없음. context_recall 평가 결과는 0이 됩니다.")
            if ANSWER_COLUMN in qa_df.columns: answer_column_exists = True
            else: print(f"  !! [경고] '{ANSWER_COLUMN}' 컬럼 없음. context_precision 등 일부 메트릭 결과에 영향 가능.")
        else:
             print("  !! [오류] Google Sheet에 헤더 외 데이터가 없습니다.")
             qa_df = pd.DataFrame() # 빈 DataFrame

    except gspread.exceptions.SpreadsheetNotFound:
         print(f"  !! [오류] Google Sheet '{GSHEET_FILE_NAME}'을(를) 찾을 수 없습니다. 파일 이름 및 공유 권한 확인 필요.")
         qa_df = pd.DataFrame()
    except Exception as e:
        print(f"  !! [오류] Google Sheet 로드 중 예상치 못한 오류: {e}")
        traceback.print_exc()
        qa_df = pd.DataFrame()
else:
    print("!! Google Sheet 클라이언트(gc)가 준비되지 않아 데이터셋을 로드할 수 없습니다.")

# --- 12.4: RAGAS 평가 데이터 준비 ---
print("\n--- 12.4: RAGAS 평가 데이터 준비 ---")
eval_data_list = []
# DataFrame이 비어있지 않고, Retriever가 준비되었을 때만 진행
if not qa_df.empty and retriever_available:
    print(f"총 {len(qa_df)}개 질문에 대해 Contexts 검색 및 데이터 형식 변환 중...")
    # tqdm으로 진행률 표시
    for index, row in tqdm(qa_df.iterrows(), total=qa_df.shape[0], desc="RAGAS 데이터 준비"):
        # 질문 추출 및 유효성 검사
        question_text = row.get(QUESTION_COLUMN)
        if pd.isna(question_text) or not str(question_text).strip():
            # print(f"  - 행 #{index}: 질문 비어있어 건너<0xEB><0x9B><0x81>니다.") # 필요 시 로그
            continue
        question_text = str(question_text).strip()

        # Answer 및 Ground Truths 추출
        answer_text = str(row.get(ANSWER_COLUMN, "")).strip() if answer_column_exists else ""
        ground_truths_input = row.get(GROUND_TRUTHS_COLUMN, "[]") if ground_truths_column_exists else "[]"

        retrieved_contexts = []
        ground_truths_list = []

        try:
            # --- Ground Truths 파싱 (문자열 -> Python 리스트) ---
            # ground_truths 컬럼이 존재하고, 내용이 문자열일 경우 파싱 시도
            if ground_truths_column_exists and isinstance(ground_truths_input, str) and ground_truths_input.strip():
                try:
                    # JSON 배열 형식("[...]")으로 가정하고 파싱
                    parsed_list = json.loads(ground_truths_input)
                    if isinstance(parsed_list, list):
                        # 리스트 내 각 항목을 문자열로 변환하고 빈 문자열 제거
                        ground_truths_list = [str(item).strip() for item in parsed_list if item is not None and str(item).strip()]
                    else: # 파싱은 됐으나 리스트 아님 (예: 단순 문자열)
                         ground_truths_list = [ground_truths_input.strip()]
                except (json.JSONDecodeError, TypeError):
                    # JSON 파싱 실패 시, 원본 문자열 자체를 하나의 ground truth로 간주
                    ground_truths_list = [ground_truths_input.strip()]
            # 이미 리스트 형태인 경우 (DataFrame에 직접 리스트가 저장된 경우)
            elif isinstance(ground_truths_input, list):
                 ground_truths_list = [str(item).strip() for item in ground_truths_input if item is not None and str(item).strip()]
            # 그 외 (비어 있거나, 숫자 등) -> 빈 리스트 유지

            # --- Retriever 실행 ---
            # 각 질문에 대해 설정된 retriever로 문서 검색
            retrieved_docs = retriever.invoke(question_text)
            # 검색된 문서들의 page_content만 리스트로 추출
            retrieved_contexts = [doc.page_content for doc in retrieved_docs]

            # --- RAGAS 평가 데이터 형식으로 구성 ---
            eval_item = {
                "question": question_text,         # RAGAS 필수 키
                "contexts": retrieved_contexts,    # RAGAS 필수 키
                "ground_truths": ground_truths_list, # RAGAS 필수 키 (Context Recall용)
                "ground_truth": answer_text        # RAGAS 선택 키 (Faithfulness, Answer Relevancy 등에서 사용)
            }
            eval_data_list.append(eval_item)

        # 데이터 준비 중 개별 항목 오류 처리
        except Exception as e:
            print(f"\n!! 데이터 준비 중 오류 (질문: '{question_text}'): {e}")
            traceback.print_exc()
            # 오류 발생 시에도 빈 데이터 추가 (전체 평가 흐름 유지)
            eval_data_list.append({
                "question": question_text, "contexts": [], "ground_truths": [], "ground_truth": ""
            })

    print(f"[완료] RAGAS 평가 데이터 {len(eval_data_list)}개 준비 완료.")
else:
    if qa_df.empty: print("!! 평가 데이터(qa_df)가 비어있거나 로드되지 않아 데이터 준비 불가.")
    if not retriever_available: print("!! Retriever가 없어 Contexts를 검색할 수 없으므로 데이터 준비 불가.")
    pass # 데이터 준비 불가

# --- 12.5: Hugging Face Dataset 형식으로 변환 ---
print("\n--- 12.5: Hugging Face Dataset 변환 ---")
eval_dataset = None
if eval_data_list:
    try:
        eval_dataset = Dataset.from_list(eval_data_list)
        print("[성공] Hugging Face Dataset으로 변환 완료.")
        print(f"   데이터셋 컬럼: {eval_dataset.column_names}")

        # === 수정된 부분: 최대 10개 샘플 출력 ===
        if len(eval_dataset) > 0:
            num_samples_to_show = min(10, len(eval_dataset)) # 최대 10개 또는 실제 데이터셋 크기
            print(f"\n   데이터셋 샘플 확인 (처음 {num_samples_to_show}개):")
            for i in range(num_samples_to_show):
                print(f"\n   --- 항목 #{i} ---") # 각 항목 구분
                sample_item = eval_dataset[i]
                for key, value in sample_item.items():
                    print(f"     {key}: ", end="")
                    preview_limit = 150 # 미리보기 글자 수
                    if isinstance(value, list):
                        # 리스트 내용 미리보기
                        preview = str(value)[:preview_limit] + ('...' if len(str(value)) > preview_limit else '')
                        print(f"(List, {len(value)} items) {preview}")
                    else:
                        # 문자열 내용 미리보기
                        preview = str(value)[:preview_limit] + ('...' if len(str(value)) > preview_limit else '')
                        print(f"{preview}")
        # === 수정 완료 ===

    except Exception as e:
        print(f"!! [오류] Dataset 변환 중 오류: {e}")
        traceback.print_exc()
        eval_dataset = None # 오류 시 None 처리
else:
     print("!! [정보] 생성된 평가 데이터가 없어 Dataset 변환을 건너<0xEB><0x9B><0x81>니다.")


# --- 12.6: RAGAS 평가 실행 ---
print("\n--- 12.6: RAGAS 평가 실행 ---")
evaluation_df = None # 평가 결과 저장 DataFrame 초기화
# 평가 실행 조건 확인
if llm_available and eval_dataset and ragas_llm and evaluate:
    # 사용할 평가 지표 정의
    metrics_to_evaluate = [context_precision, context_recall]
    print(f"평가 지표: {[m.name for m in metrics_to_evaluate if m]}") # None 아닌 메트릭 이름만 출력

    # 필요한 메트릭이 모두 준비되었는지 확인
    if not all(metrics_to_evaluate):
        print("!! [오류] 평가에 필요한 Ragas 메트릭(context_precision, context_recall) 임포트 실패.")
    else:
        try:
            # (참고) LLM Timeout/Retry 설정은 필요시 LangChain LLM 객체 생성 시점에 하는 것이 더 안정적일 수 있음
            print("  (LLM Timeout/Retry는 Langchain LLM 객체 생성 시 설정 권장)")

            # RAGAS 평가 실행
            print(f"RAGAS 평가 실행 (데이터 {len(eval_dataset)}개, 배치 크기 2)...")
            # evaluate 함수 호출
            result_ragas = evaluate(
                dataset=eval_dataset,           # Hugging Face Dataset
                metrics=metrics_to_evaluate,    # 평가 지표 리스트
                llm=ragas_llm,                  # Ragas LLM 래퍼
                embeddings=ragas_embeddings if embeddings_available else None, # Ragas Embedding 래퍼 (없으면 None)
                batch_size=2                    # 배치 크기 (API 제한 고려하여 조절)
                # is_async=False # 기본값은 True이나, Colab 등 환경따라 False가 안정적일 수 있음
            )
            evaluation_df = result_ragas.to_pandas() # 결과를 pandas DataFrame으로
            print("[성공] RAGAS 평가 완료.")

            # --- 개별 질문별 결과 출력 ---
            print("\n--- 개별 질문별 평가 결과 ---")
            for index, row in evaluation_df.iterrows():
                question = eval_dataset[index].get("question", f"질문 #{index+1}")
                print(f"\n**질문 {index + 1}:** {question}")
                for metric in metrics_to_evaluate:
                    metric_name = metric.name
                    if metric_name in row.index and not pd.isna(row[metric_name]):
                        print(f"  - {metric_name}: {row[metric_name]:.4f}") # 소수점 4자리
                    else:
                        print(f"  - {metric_name}: N/A (계산 실패)")
                print("-" * 30)

        except Exception as e:
            print(f"!! [오류] RAGAS 평가 중 심각한 오류 발생: {e}")
            print("   - LLM API 호출 실패(Timeout, 할당량 초과 등) 가능성이 높습니다.")
            print("   - Google Cloud Console API 할당량 확인 또는 batch_size=1 시도 권장.")
            traceback.print_exc()
else:
    # 평가 실행 불가 사유 요약
    missing_components = []
    if not llm_available: missing_components.append("LLM")
    if not eval_dataset: missing_components.append("평가 데이터셋")
    if not ragas_llm: missing_components.append("RAGAS LLM 래퍼")
    if not evaluate: missing_components.append("RAGAS evaluate 함수")
    print(f"!! {', '.join(missing_components)} 준비 안 됨. RAGAS 평가 불가.")

# --- 12.7: 전체 평가 결과 요약 ---
print("\n--- 12.7: RAGAS Retriever 전체 평가 결과 요약 ---")
if evaluation_df is not None:
    # 평가된 메트릭 컬럼만 필터링
    existing_metrics = [m.name for m in metrics_to_evaluate if m and m.name in evaluation_df.columns]
    if existing_metrics:
        print("\n=== 전체 결과 (요약) ===")
        print(evaluation_df[existing_metrics].round(4).to_string(na_rep='NaN')) # NaN 값 표시

        # NaN 제외 평균 계산
        avg_scores = evaluation_df[existing_metrics].mean(skipna=True)
        print("\n=== 지표별 평균 점수 (NaN 제외) ===")
        print(avg_scores.round(4))

        # NaN 발생 요약
        nan_counts = evaluation_df[existing_metrics].isna().sum()
        if nan_counts.sum() > 0:
             print("\n[참고] 일부 항목 NaN 발생:")
             print(nan_counts[nan_counts > 0])
             print("  (LLM 응답 오류, Timeout 등으로 점수 계산 불가 시 발생 가능)")
    else:
        print("\n!! evaluation_df에 표시할 메트릭 컬럼이 없습니다.")

    print(f"\n**참고:**")
    print(f"  - 'context_recall': Google Sheet '{GROUND_TRUTHS_COLUMN}' 컬럼 기준.")
    print(f"  - 'context_precision': '{QUESTION_COLUMN}', '{ANSWER_COLUMN}' 컬럼 참조 가능.")
else:
    print("RAGAS 평가가 실행되지 않았거나 오류가 발생하여 요약할 결과가 없습니다.")

print("\n--- 단계 12: Retriever 성능 평가 (RAGAS) 완료 ---")


--- 단계 12: RAGAS Retriever 평가 시작 ---
[정보] RAGAS 및 관련 라이브러리 임포트 확인.

--- 12.1: Google 인증 및 gspread 클라이언트 확인 ---
[성공] Google 인증 및 gspread 클라이언트 준비 완료.

--- 12.2: 필수 객체 확인 및 RAGAS 래퍼 초기화 ---
[성공] RAGAS LLM 래퍼 초기화 완료 (모델: models/gemini-1.5-flash)
[정보] Retriever 객체('retriever') 확인 완료.
[성공] RAGAS Embedding 래퍼 초기화 완료.

--- 12.3: 평가 데이터셋 로드 ---
Google Sheet 'filtered_qa_dataset' 로드 시도...
  워크시트 로드 중...
  [성공] Google Sheet 로드 완료: 89개 행
     컬럼 목록: ['question', 'answer', 'ground_truths']

--- 12.4: RAGAS 평가 데이터 준비 ---
총 89개 질문에 대해 Contexts 검색 및 데이터 형식 변환 중...


RAGAS 데이터 준비:   0%|          | 0/89 [00:00<?, ?it/s]

[완료] RAGAS 평가 데이터 89개 준비 완료.

--- 12.5: Hugging Face Dataset 변환 ---
[성공] Hugging Face Dataset으로 변환 완료.
   데이터셋 컬럼: ['question', 'contexts', 'ground_truths', 'ground_truth']

   데이터셋 샘플 확인 (처음 10개):

   --- 항목 #0 ---
     question: 근로계약이 미성년자에게 불리하다고 인정되는 경우 미성년후견인은 그 계약을 해지할 수 있나요?
     contexts: (List, 10 items) ['법제처                                                            23                                                       국가법령정보센터\n국민건강보험법 시행규칙\n② 제1...
     ground_truths: (List, 1 items) ['["「근로기준법」 제67조 제2항은 "친권자","후견인 또는 고용노동부장관은 근로계약이 미성년자에게 불리하다고 인정하는 경우에는 이를 해지할 수 있다."라고 규정하고 있습니다."]']
     ground_truth: 네. 근로계약을 해지할 수 있습니다.

   --- 항목 #1 ---
     question: 업무상 재해로 요양이 끝나고 복직 신청을 한 근로자가 복직후 3일도 안되어 중대한 사고를 발생시켜, 징계위원회를 거쳐 해고하고자 합니다. 그런데, 근로자가 회사로 부터 해고예고통지서를 받았음에도 불구하고 재요양을 신청하여 휴직중입니다. 복직 후 다시 해고예고 하여야 하...
     contexts: (List, 10 items) ['법제처                                                            5                                                       국가법령정보센터\n

Evaluating:   0%|          | 0/178 [00:00<?, ?it/s]

Batch 1/89:   0%|          | 0/2 [00:00<?, ?it/s]

}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 18
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 11
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 8
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 53
}
].
}
, links {
  description: "Learn more abo

KeyboardInterrupt: 

# 13: RAGAS 생성된 답변 평가

In [16]:
# === 단계 13: RAGAS 평가 (생성된 답변 대상) ===
# 이 블록은 단계 11에서 RAG 시스템 실행(qa_chain.invoke) 후 실행해야 합니다.

print("\n--- 단계 12: RAGAS 평가 시작 (Faithfulness, Answer Relevancy) ---")

# 필요한 라이브러리 임포트 (이전 블록에서 임포트했더라도 명시적으로 다시 확인)
import warnings
from datasets import Dataset
# RAGAS 관련 임포트 (오류 발생 시 메시지 출력하며 None으로 설정)
try:
    from ragas import evaluate
    from ragas.metrics import faithfulness, answer_relevancy
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper
    print("[정보] RAGAS 및 관련 라이브러리 임포트 확인.")
except ImportError as e:
    print(f"!! [오류] RAGAS 관련 라이브러리 임포트 실패: {e}")
    print("   이전 단계의 라이브러리 설치가 성공했는지, 런타임 재시작이 필요한지 확인하세요.")
    evaluate = None; faithfulness = None; answer_relevancy = None;
    LangchainLLMWrapper = None; LangchainEmbeddingsWrapper = None

warnings.filterwarnings("ignore") # 경고 메시지 숨기기

# --- 13.1: 평가에 필요한 객체 및 데이터 확인 ---
print("\n--- 12.1: 평가 필요 객체 확인 ---")

# 이전 단계(Step 11)에서 생성된 결과 확인
rag_execution_successful = ('query' in locals() and query and
                           'result' in locals() and isinstance(result, dict) and
                           'result' in result and 'source_documents' in result)

# 평가에 필요한 LLM 및 Embedding 모델 확인 (이전 단계에서 정의됨)
llm_available = 'llm' in locals() and llm is not None
embeddings_available = 'embeddings' in locals() and embeddings is not None

if not rag_execution_successful:
    print("!! [오류] 이전 단계(단계 11)의 RAG 실행 결과('query', 'result') 없음. 평가 불가.")
if not llm_available:
    print("!! [오류] 평가용 LLM 객체('llm') 없음. 평가 불가.")
if not embeddings_available:
    print("!! [경고] 평가용 Embedding 객체('embeddings') 없음. 'answer_relevancy' 계산 불가.")

# 모든 필수 요소가 준비되었는지 최종 확인
ready_for_evaluation = (rag_execution_successful and llm_available and embeddings_available and
                        evaluate and faithfulness and answer_relevancy and
                        LangchainLLMWrapper and LangchainEmbeddingsWrapper) # RAGAS 함수/클래스 임포트 확인

# --- 13.2: RAGAS 평가 데이터 준비 ---
eval_dataset = None
if ready_for_evaluation:
    print("\n--- 12.2: RAGAS 평가 데이터 준비 ---")
    try:
        # Step 11의 결과를 RAGAS가 요구하는 형식으로 변환
        eval_data = {
            'question': [query],  # Step 11의 질문 (리스트 형태)
            'answer': [result['result']], # Step 11의 생성된 답변 (리스트 형태)
            'contexts': [[doc.page_content for doc in result['source_documents']]], # 검색된 문서 내용 리스트 (리스트의 리스트 형태)
            # 'ground_truth': ["여기에 이상적인 정답 문자열을 넣을 수 있습니다"] # (선택 사항) 만약 이상적인 정답이 있다면 추가
        }
        eval_dataset = Dataset.from_dict(eval_data)
        print("[성공] 평가용 데이터셋 생성 완료 (1개 항목).")
        # print("  샘플 데이터:", eval_dataset[0]) # 디버깅용
    except Exception as e:
        print(f"!! [오류] 평가 데이터셋 생성 중 오류: {e}")
        eval_dataset = None
        ready_for_evaluation = False # 데이터 준비 실패 시 평가 불가

# --- 13.3: RAGAS 평가 실행 ---
evaluation_results = None
if ready_for_evaluation:
    print("\n--- 12.3: RAGAS 평가 실행 ---")
    try:
        # RAGAS Metric 정의 (이 평가에서는 답변 품질 지표만 사용)
        metrics_to_evaluate = [
            faithfulness,     # 답변이 컨텍스트에 얼마나 충실한가 (LLM 필요)
            answer_relevancy  # 답변이 질문과 얼마나 관련 있는가 (LLM 및 Embeddings 필요)
        ]
        print(f"평가 지표: {[m.name for m in metrics_to_evaluate]}")

        # RAGAS 래퍼 초기화 (이전 Ragas 평가 블록과 유사)
        # LLM 래퍼
        ragas_llm_eval = LangchainLLMWrapper(llm)
        # Embedding 래퍼
        ragas_embeddings_eval = LangchainEmbeddingsWrapper(embeddings)

        # 평가 실행
        print("RAGAS 평가 실행 중...")
        evaluation_results = evaluate(
            dataset=eval_dataset,
            metrics=metrics_to_evaluate,
            llm=ragas_llm_eval,
            embeddings=ragas_embeddings_eval # answer_relevancy에 필요
            # is_async=False # Colab 등에서 동시성 문제 발생 시 False 시도
        )
        print("[성공] RAGAS 평가 완료.")

    except Exception as e:
        print(f"!! [오류] RAGAS 평가 실행 중 오류 발생: {e}")
        print("   - API 키 유효성, 할당량(Rate Limit), 네트워크 연결 확인 필요.")
        print("   - LLM/Embedding 모델 호환성 또는 RAGAS 버전 문제일 수 있음.")
        traceback.print_exc() # 상세 오류 스택 출력
        evaluation_results = None
else:
    print("\n!! 필수 요소 부족으로 RAGAS 평가를 건너<0xEB><0x9B><0x81>니다.")


# --- 13.4: 평가 결과 출력 ---
print("\n--- 12.4: RAGAS 평가 결과 ---")
if evaluation_results:
    # Ragas 결과는 딕셔너리 형태일 수 있음 (최신 버전 확인 필요)
    if isinstance(evaluation_results, dict):
        print("{")
        for metric_name, score in evaluation_results.items():
             # NaN 값 처리 추가
             score_str = f"{score:.4f}" if isinstance(score, (int, float)) and not np.isnan(score) else "NaN"
             print(f"  '{metric_name}': {score_str},")
        print("}")
    else:
         # DataFrame 등으로 반환될 경우 처리
         print(evaluation_results)

    print("\n**지표 설명:**")
    print("  - faithfulness: 답변이 제공된 컨텍스트 문서 내용에 얼마나 기반하는지 (높을수록 좋음,幻覺 적음)")
    print("  - answer_relevancy: 생성된 답변이 원본 질문과 얼마나 관련성이 높은지 (높을수록 좋음)")
else:
    print("평가 결과가 없습니다.")

print("\n--- 단계 12: RAGAS 평가 완료 ---")


--- 단계 12: RAGAS 평가 시작 (Faithfulness, Answer Relevancy) ---
[정보] RAGAS 및 관련 라이브러리 임포트 확인.

--- 12.1: 평가 필요 객체 확인 ---

--- 12.2: RAGAS 평가 데이터 준비 ---
[성공] 평가용 데이터셋 생성 완료 (1개 항목).

--- 12.3: RAGAS 평가 실행 ---
평가 지표: ['faithfulness', 'answer_relevancy']
RAGAS 평가 실행 중...


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

[성공] RAGAS 평가 완료.

--- 12.4: RAGAS 평가 결과 ---
{'faithfulness': 1.0000, 'answer_relevancy': 0.8728}

**지표 설명:**
  - faithfulness: 답변이 제공된 컨텍스트 문서 내용에 얼마나 기반하는지 (높을수록 좋음,幻覺 적음)
  - answer_relevancy: 생성된 답변이 원본 질문과 얼마나 관련성이 높은지 (높을수록 좋음)

--- 단계 12: RAGAS 평가 완료 ---


# 14: 정성적 평가 (LLM-as-Judge)

In [17]:
# -*- coding: utf-8 -*-
# === 단계 14: 정성적 평가 (LLM-as-Judge) ===
# (ChatOpenAI 사용에 맞게 수정됨)

# 이 블록은 단계 11에서 RAG 시스템 실행(qa_chain.invoke 또는 __call__) 후 실행해야 합니다.

print("\n--- 단계 14: 정성적 평가 (LLM-as-Judge) 시작 ---") # <<< 단계 번호 수정

# --- 14.1: 필요한 라이브러리 및 객체 확인 ---
print("\n--- 14.1: 필요한 라이브러리 및 객체 확인 ---") # <<< 단계 번호 수정
import json
import traceback
import re # JSON 추출 위해 임포트 추가
# LangChain 관련 (이미 임포트 되었을 가능성 높음, 확인 차원)
try: from langchain.prompts import PromptTemplate
except ImportError: print("!! [오류] langchain.prompts 임포트 실패")
# LLM 클래스 확인 (ChatOpenAI)
try: from langchain_openai import ChatOpenAI
except ImportError: print("!! [오류] langchain_openai.ChatOpenAI 임포트 실패")

# 이전 단계 결과 확인
rag_execution_successful = ('query' in locals() and query and
                           'result' in locals() and isinstance(result, dict) and
                           'result' in result and 'source_documents' in result)
# 평가자 LLM 확인 (단계 9에서 정의된 llm 사용)
llm_available = 'llm' in locals() and llm is not None and isinstance(llm, ChatOpenAI) # 타입 확인 추가

if not rag_execution_successful:
    print("!! [오류] 이전 단계(단계 11)의 RAG 실행 결과('query', 'result') 없음. 평가 불가.")
if not llm_available:
    print("!! [오류] 평가용 LLM 객체('llm')가 없거나 ChatOpenAI 타입이 아님. 평가 불가.")

# --- 14.2: 평가 실행 ---
print("\n--- 14.2: 평가 실행 ---") # <<< 단계 번호 수정
if rag_execution_successful and llm_available:
    # 평가에 사용할 LLM (기존 llm 사용)
    evaluator_llm = llm
    # 만약 다른 강력한 모델을 쓰고 싶다면 여기서 재정의
    # evaluator_llm = ChatOpenAI(model="gpt-4o", temperature=0)

    # ====> 수정된 부분: .model 대신 .model_name 사용 <====
    print(f"[정보] 평가자 LLM: {evaluator_llm.model_name}")
    # ===============================================

    # 평가 데이터 추출
    rag_answer = result.get("result", "")
    source_docs = result.get("source_documents", []) # source_documents가 없다면 이전 단계 문제
    if not source_docs:
         print("!! [경고] result 객체에 'source_documents'가 비어있거나 없습니다. 컨텍스트 없는 평가 진행.")
         context_str = "" # 컨텍스트 없이 진행
    else:
         context_str = "\n\n".join([doc.page_content for doc in source_docs])

    # 평가 프롬프트 템플릿 (기존 유지)
    evaluation_template = """당신은 AI 응답 품질 평가 전문가입니다. 주어진 '질문', '컨텍스트 문서', 그리고 AI가 생성한 '답변'을 바탕으로 다음 기준에 따라 답변의 품질을 평가해주세요. 평가는 '컨텍스트 문서'에 있는 내용만을 근거로 해야 합니다.

    [평가 기준]
    1.  **Faithfulness (충실성):** 답변이 '컨텍스트 문서' 내용에만 기반하고 있습니까? 외부 정보를 추가하거나 컨텍스트와 모순되는 내용은 없습니까? (Yes/No)
    2.  **Relevance (관련성):** 답변이 사용자의 '질문'과 직접적으로 관련이 있습니까? (Yes/No)
    3.  **Clarity (명확성):** 답변이 명확하고 간결하며 이해하기 쉽습니까? (Yes/No)
    4.  **Completeness (완전성 - 컨텍스트 기준):** 답변이 '컨텍스트 문서' 내에서 찾을 수 있는 정보 범위 내에서 '질문'에 대해 충분히 완전하게 답변하고 있습니까? (Yes/No)

    [입력 정보]
    질문: {question}

    컨텍스트 문서:
    {context}

    AI 생성 답변: {answer}

    [평가 결과 출력]
    각 평가 기준에 대해 "Yes" 또는 "No"로 평가하고, 각 평가에 대한 간략한 근거를 포함하여 JSON 형식으로 결과를 반환해주세요.
    예시: {{
        "faithfulness": {{\"score\": \"Yes\", \"justification\": \"답변 내용이 모두 컨텍스트 문서에서 확인되었습니다.\"}},
        "relevance": {{\"score\": \"Yes\", \"justification\": \"질문의 핵심 내용을 정확히 파악하고 답변했습니다.\"}},
        "clarity": {{\"score\": \"Yes\", \"justification\": \"간결하고 명확한 문장으로 작성되었습니다.\"}},
        "completeness": {{\"score\": \"No\", \"justification\": \"질문의 일부 측면(예: 예외 사항)에 대한 정보가 컨텍스트에 있었으나 답변에 포함되지 않았습니다.\"}}
    }}

    JSON 출력:
    """
    try:
        EVALUATION_PROMPT = PromptTemplate.from_template(evaluation_template)
    except NameError:
         print("!! [오류] PromptTemplate 클래스를 찾을 수 없습니다. 단계 1 임포트를 확인하세요.")
         EVALUATION_PROMPT = None

    # 평가 체인 구성 및 실행
    if EVALUATION_PROMPT:
        try:
            print("\nLLM 기반 정성적 평가 진행 중...")
            eval_chain = EVALUATION_PROMPT | evaluator_llm # LangChain Expression Language 사용

            # 컨텍스트 길이 제한 (평가자 LLM의 토큰 제한 고려)
            # GPT-4o는 긴 컨텍스트 처리 가능 (예: 128k), 비용 및 효율성 위해 제한 가능
            max_context_length = 100000 # 예시: 필요시 조정 (GPT-4o에 맞춰 늘림)
            if len(context_str) > max_context_length:
                print(f"  [경고] 컨텍스트 길이가 길어 {max_context_length}자로 잘라냅니다.")
                context_for_eval = context_str[:max_context_length]
            else:
                context_for_eval = context_str
                if not context_for_eval: # source_documents 없을 경우
                    print("  [정보] 컨텍스트 문서 없이 평가 진행.")

            # 입력값 준비
            chain_input = {
                "question": query if query else "질문 정보 없음", # query 변수 확인
                "context": context_for_eval,
                "answer": rag_answer if rag_answer else "답변 정보 없음" # rag_answer 변수 확인
            }

            eval_result_str = eval_chain.invoke(chain_input).content

            print("\n[LLM 평가 결과 (Raw)]:\n", eval_result_str)

            # JSON 파싱 시도 및 결과 출력
            print("\n[파싱된 평가 결과]:")
            try:
                # LLM 응답에서 JSON 부분만 추출 (마크다운 ```json ... ``` 고려)
                match = re.search(r'```json\s*(\{.*?\})\s*```', eval_result_str, re.DOTALL | re.IGNORECASE)
                if match:
                    json_str = match.group(1)
                else:
                    # 코드 블록 없으면, 응답 시작이 '{' 인지 확인
                    json_str = eval_result_str.strip()
                    # 가끔 LLM이 JSON 앞에 불필요한 텍스트를 붙일 수 있으므로, '{' 를 찾아 시작점 조정 시도
                    start_brace = json_str.find('{')
                    if start_brace != -1:
                        json_str = json_str[start_brace:]
                        # 마지막 '}' 이후 내용 제거 시도 (단순 구현)
                        end_brace = json_str.rfind('}')
                        if end_brace != -1:
                             json_str = json_str[:end_brace+1]
                        else:
                             raise json.JSONDecodeError("닫는 중괄호 '}' 없음", json_str, 0)
                    else:
                         raise json.JSONDecodeError("응답이 JSON 형식으로 시작하지 않음", json_str, 0)

                eval_data = json.loads(json_str)
                for criterion, details in eval_data.items():
                    # details가 딕셔너리 형태인지 확인
                    if isinstance(details, dict):
                         print(f"- {str(criterion).capitalize()}:") # 키가 문자열 아닐 수도 있으니 str() 추가
                         print(f"  - 점수: {details.get('score', 'N/A')}")
                         print(f"  - 근거: {details.get('justification', 'N/A')}")
                    else:
                         # 예상치 못한 형식일 경우 그대로 출력
                         print(f"- {str(criterion).capitalize()}: {details}")

            except json.JSONDecodeError as json_e:
                print(f"  !! 평가 결과가 유효한 JSON 형식이 아니거나 파싱 오류 발생: {json_e}")
                print("     Raw 결과를 직접 확인하세요.")
            except Exception as parse_e:
                print(f"  !! 평가 결과 처리 중 오류: {parse_e}")
                traceback.print_exc()

        except Exception as e:
            print(f"!! LLM 평가 실행 중 오류 발생: {e}")
            print("   - API 키, 할당량, 네트워크 연결 등을 확인하세요.")
            traceback.print_exc()
    else:
        print("!! [오류] 평가 프롬프트 템플릿(EVALUATION_PROMPT) 생성 실패.")

else:
    # 평가 실행 불가 사유 출력
    print("\n!! 이전 단계 결과 또는 LLM 객체가 준비되지 않아 정성적 평가를 진행할 수 없습니다.")


print("\n--- 단계 14: 정성적 평가 완료 ---") # <<< 단계 번호 수정


--- 단계 14: 정성적 평가 (LLM-as-Judge) 시작 ---

--- 14.1: 필요한 라이브러리 및 객체 확인 ---

--- 14.2: 평가 실행 ---
[정보] 평가자 LLM: gpt-4.1-nano

LLM 기반 정성적 평가 진행 중...

[LLM 평가 결과 (Raw)]:
 {
    "faithfulness": {
        "score": "Yes",
        "justification": "답변이 컨텍스트 문서의 법령 조항 내용을 그대로 인용하고 있으며, 외부 정보나 모순되는 내용이 없습니다."
    },
    "relevance": {
        "score": "Yes",
        "justification": "질문이 근로자를 해고하려면 몇일 전에 예고를 해야 하는지에 관한 것이므로, 답변이 그 질문에 직접적으로 관련되어 있습니다."
    },
    "clarity": {
        "score": "Yes",
        "justification": "답변이 간결하고 명확하게 근로기준법 제26조 내용을 설명하고 있어 이해하기 쉽습니다."
    },
    "completeness": {
        "score": "No",
        "justification": "컨텍스트 문서에는 해고 예고와 관련된 예외 사유(예: 해고 시기, 특정 조건 등)가 언급되어 있으나, 답변은 단순히 법령 조항만을 인용하여 예외 사항이나 구체적 상황에 대한 설명이 부족하여 질문에 대한 완전한 답변은 아닙니다."
    }
}

[파싱된 평가 결과]:
- Faithfulness:
  - 점수: Yes
  - 근거: 답변이 컨텍스트 문서의 법령 조항 내용을 그대로 인용하고 있으며, 외부 정보나 모순되는 내용이 없습니다.
- Relevance:
  - 점수: Yes
  - 근거: 질문이 근로자를 해고하려면 몇일 전에 예고를 해야 하는지에 관한 것이므로, 답변이 그 질문에 직접적으로 관련되어 있습니다.
- Clar